<a href="https://colab.research.google.com/github/AaronAM/ZeroERP/blob/main/ZRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%writefile ZeroERP.js
import React, { useState, useMemo } from 'react';
import {
  LayoutDashboard,
  Package,
  ShoppingCart,
  Truck,
  Settings,
  Search,
  Bell,
  Menu,
  X,
  ArrowUpRight,
  ArrowDownRight,
  Plus,
  Filter,
  Download,
  CheckCircle,
  AlertCircle,
  DollarSign,
  BarChart3
} from 'lucide-react';

// --- Mock Data Generators ---

const INITIAL_INVENTORY = [
  { id: 'SKU-001', name: 'Ceramic Pour-Over Set', category: 'Coffee', stock: { warehouse: 45, store: 12 }, safetyStock: 20, cost: 15.00, price: 45.00, vendor: 'CeramicsCo' },
  { id: 'SKU-002', name: 'Ethiopian Yirgacheffe (12oz)', category: 'Coffee', stock: { warehouse: 120, store: 50 }, safetyStock: 50, cost: 8.50, price: 22.00, vendor: 'GlobalBeans' },
  { id: 'SKU-003', name: 'Gooseneck Kettle (Matte Black)', category: 'Equipment', stock: { warehouse: 5, store: 2 }, safetyStock: 10, cost: 35.00, price: 85.00, vendor: 'HeatSync' },
  { id: 'SKU-004', name: 'Paper Filters (100ct)', category: 'Consumables', stock: { warehouse: 500, store: 100 }, safetyStock: 200, cost: 2.00, price: 8.00, vendor: 'PaperWorks' },
  { id: 'SKU-005', name: 'Barista Apron (Canvas)', category: 'Apparel', stock: { warehouse: 25, store: 5 }, safetyStock: 15, cost: 12.00, price: 40.00, vendor: 'TextileBros' },
  { id: 'SKU-006', name: 'Cold Brew Kit', category: 'Bundles', stock: { warehouse: 15, store: 0 }, safetyStock: 10, cost: 18.00, price: 55.00, vendor: 'Internal' },
];

const INITIAL_ORDERS = [
  { id: 'ORD-7782', customer: 'Alice Freeman', channel: 'Shopify', total: 135.00, status: 'Pending', date: '2023-10-24', items: 3 },
  { id: 'ORD-7781', customer: 'Mark Stone', channel: 'Amazon', total: 22.00, status: 'Shipped', date: '2023-10-24', items: 1 },
  { id: 'ORD-7780', customer: 'Cafe Lux (Wholesale)', channel: 'B2B Portal', total: 450.00, status: 'Shipped', date: '2023-10-23', items: 20 },
  { id: 'ORD-7779', customer: 'Sarah Connor', channel: 'Shopify', total: 85.00, status: 'Delivered', date: '2023-10-23', items: 1 },
];

const INITIAL_PO = [
  { id: 'PO-9001', vendor: 'HeatSync', status: 'Ordered', expected: '2023-11-01', total: 1500.00 },
  { id: 'PO-9002', vendor: 'GlobalBeans', status: 'Received', expected: '2023-10-20', total: 850.00 },
];

// --- Components ---

const Card = ({ children, className = "" }) => (
  <div className={`bg-white rounded-xl border border-slate-200 shadow-sm ${className}`}>
    {children}
  </div>
);

const Badge = ({ status }) => {
  const styles = {
    Pending: "bg-amber-100 text-amber-700",
    Shipped: "bg-blue-100 text-blue-700",
    Delivered: "bg-emerald-100 text-emerald-700",
    Ordered: "bg-purple-100 text-purple-700",
    Received: "bg-slate-100 text-slate-700",
    Low: "bg-red-100 text-red-700",
    Normal: "bg-emerald-100 text-emerald-700",
  };
  return (
    <span className={`px-2.5 py-0.5 rounded-full text-xs font-medium ${styles[status] || "bg-gray-100 text-gray-800"}`}>
      {status}
    </span>
  );
};

const MetricCard = ({ title, value, subtext, trend, trendUp }) => (
  <Card className="p-5">
    <div className="flex justify-between items-start">
      <div>
        <p className="text-sm font-medium text-slate-500">{title}</p>
        <h3 className="text-2xl font-bold text-slate-900 mt-1">{value}</h3>
      </div>
      <div className={`p-2 rounded-lg ${trendUp ? 'bg-emerald-50' : 'bg-rose-50'}`}>
        {trendUp ? <ArrowUpRight className="w-5 h-5 text-emerald-600" /> : <ArrowDownRight className="w-5 h-5 text-rose-600" />}
      </div>
    </div>
    <div className="mt-4 flex items-center text-xs">
      <span className={`font-medium ${trendUp ? 'text-emerald-600' : 'text-rose-600'}`}>
        {trend}
      </span>
      <span className="text-slate-400 ml-1">{subtext}</span>
    </div>
  </Card>
);

// --- Main Application ---

export default function App() {
  const [activeTab, setActiveTab] = useState('dashboard');
  const [isMobileMenuOpen, setIsMobileMenuOpen] = useState(false);

  // App State
  const [inventory, setInventory] = useState(INITIAL_INVENTORY);
  const [orders, setOrders] = useState(INITIAL_ORDERS);
  const [purchaseOrders, setPurchaseOrders] = useState(INITIAL_PO);
  const [searchTerm, setSearchTerm] = useState('');

  // Derived Metrics
  const totalStockValue = useMemo(() =>
    inventory.reduce((acc, item) => acc + ((item.stock.warehouse + item.stock.store) * item.cost), 0),
  [inventory]);

  const lowStockCount = useMemo(() =>
    inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).length,
  [inventory]);

  const pendingOrdersCount = useMemo(() =>
    orders.filter(o => o.status === 'Pending').length,
  [orders]);

  // Actions
  const fulfillOrder = (orderId) => {
    setOrders(orders.map(o => o.id === orderId ? { ...o, status: 'Shipped' } : o));
    // In a real app, this would decrement inventory
    alert(`Order ${orderId} marked as Shipped. (Simulation: Inventory would be adjusted here)`);
  };

  const receivePO = (poId) => {
    setPurchaseOrders(purchaseOrders.map(p => p.id === poId ? { ...p, status: 'Received' } : p));
    alert(`PO ${poId} received. (Simulation: Stock levels would be updated here)`);
  };

  const filteredInventory = inventory.filter(item =>
    item.name.toLowerCase().includes(searchTerm.toLowerCase()) ||
    item.id.toLowerCase().includes(searchTerm.toLowerCase())
  );

  const Navigation = () => (
    <nav className="space-y-1">
      {[
        { id: 'dashboard', label: 'Dashboard', icon: LayoutDashboard },
        { id: 'inventory', label: 'Inventory', icon: Package },
        { id: 'orders', label: 'Orders', icon: ShoppingCart },
        { id: 'purchasing', label: 'Purchasing', icon: Truck },
        { id: 'reports', label: 'Reports', icon: BarChart3 },
        { id: 'settings', label: 'Settings', icon: Settings },
      ].map((item) => (
        <button
          key={item.id}
          onClick={() => {
            setActiveTab(item.id);
            setIsMobileMenuOpen(false);
          }}
          className={`w-full flex items-center px-4 py-3 text-sm font-medium rounded-lg transition-colors ${
            activeTab === item.id
              ? 'bg-slate-900 text-white'
              : 'text-slate-600 hover:bg-slate-100'
          }`}
        >
          <item.icon className="w-5 h-5 mr-3" />
          {item.label}
          {item.id === 'orders' && pendingOrdersCount > 0 && (
            <span className="ml-auto bg-rose-500 text-white text-xs font-bold px-2 py-0.5 rounded-full">
              {pendingOrdersCount}
            </span>
          )}
        </button>
      ))}
    </nav>
  );

  const renderContent = () => {
    switch(activeTab) {
      case 'dashboard':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4">
              <MetricCard
                title="Total Stock Value"
                value={`$${totalStockValue.toLocaleString()}`}
                trend="+12%"
                subtext="vs last month"
                trendUp={true}
              />
              <MetricCard
                title="Pending Orders"
                value={pendingOrdersCount}
                trend="-5%"
                subtext="processing faster"
                trendUp={true}
              />
              <MetricCard
                title="Low Stock SKUs"
                value={lowStockCount}
                trend="+2"
                subtext="needs attention"
                trendUp={false}
              />
              <MetricCard
                title="Gross Margin (Avg)"
                value="68%"
                trend="+1.2%"
                subtext="healthy"
                trendUp={true}
              />
            </div>

            <div className="grid grid-cols-1 lg:grid-cols-3 gap-6">
              <Card className="col-span-2 p-6">
                <div className="flex justify-between items-center mb-6">
                  <h3 className="font-bold text-slate-800">Recent Sales Velocity</h3>
                  <select className="text-sm border-slate-200 rounded-md text-slate-500">
                    <option>Last 7 Days</option>
                  </select>
                </div>
                {/* Simulated Chart */}
                <div className="h-64 flex items-end space-x-2">
                  {[45, 60, 35, 70, 85, 60, 75].map((h, i) => (
                    <div key={i} className="flex-1 bg-slate-100 rounded-t-lg relative group">
                      <div
                        className="absolute bottom-0 w-full bg-slate-800 rounded-t-lg transition-all duration-500 hover:bg-indigo-600"
                        style={{ height: `${h}%` }}
                      ></div>
                      <div className="opacity-0 group-hover:opacity-100 absolute -top-8 left-1/2 -translate-x-1/2 bg-slate-900 text-white text-xs py-1 px-2 rounded">
                        ${h * 100}
                      </div>
                    </div>
                  ))}
                </div>
                <div className="flex justify-between mt-2 text-xs text-slate-400">
                  <span>Mon</span><span>Tue</span><span>Wed</span><span>Thu</span><span>Fri</span><span>Sat</span><span>Sun</span>
                </div>
              </Card>

              <Card className="p-6">
                <h3 className="font-bold text-slate-800 mb-4">Stock Alerts</h3>
                <div className="space-y-4">
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).map(item => (
                    <div key={item.id} className="flex items-start p-3 bg-red-50 rounded-lg border border-red-100">
                      <AlertCircle className="w-5 h-5 text-red-500 mt-0.5 mr-3 flex-shrink-0" />
                      <div>
                        <p className="text-sm font-medium text-red-900">{item.name}</p>
                        <p className="text-xs text-red-700 mt-1">
                          Only {item.stock.warehouse + item.stock.store} units left (Safety: {item.safetyStock})
                        </p>
                        <button className="mt-2 text-xs font-medium text-red-600 hover:text-red-800">
                          Create PO &rarr;
                        </button>
                      </div>
                    </div>
                  ))}
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) >= i.safetyStock).slice(0, 2).map(item => (
                    <div key={item.id} className="flex items-center p-3 hover:bg-slate-50 rounded-lg transition-colors">
                      <div className="w-2 h-2 rounded-full bg-emerald-500 mr-3"></div>
                      <div className="flex-1">
                        <p className="text-sm font-medium text-slate-700">{item.name}</p>
                        <p className="text-xs text-slate-500">{item.stock.warehouse + item.stock.store} Units Available</p>
                      </div>
                    </div>
                  ))}
                </div>
              </Card>
            </div>
          </div>
        );

      case 'inventory':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex flex-col sm:flex-row justify-between items-start sm:items-center gap-4">
              <h2 className="text-xl font-bold text-slate-900">Inventory Management</h2>
              <div className="flex gap-2">
                <button className="flex items-center px-4 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50">
                  <Download className="w-4 h-4 mr-2" /> Export
                </button>
                <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                  <Plus className="w-4 h-4 mr-2" /> Add SKU
                </button>
              </div>
            </div>

            <Card className="overflow-hidden">
              <div className="p-4 border-b border-slate-100 bg-slate-50/50 flex flex-col sm:flex-row gap-4">
                <div className="relative flex-1">
                  <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
                  <input
                    type="text"
                    placeholder="Search SKUs, names, or categories..."
                    className="w-full pl-10 pr-4 py-2 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
                    value={searchTerm}
                    onChange={(e) => setSearchTerm(e.target.value)}
                  />
                </div>
                <button className="flex items-center px-3 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50">
                  <Filter className="w-4 h-4 mr-2" /> Filters
                </button>
              </div>

              <div className="overflow-x-auto">
                <table className="w-full text-left text-sm">
                  <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                    <tr>
                      <th className="px-6 py-4">Product / SKU</th>
                      <th className="px-6 py-4">Location Split</th>
                      <th className="px-6 py-4 text-right">Cost</th>
                      <th className="px-6 py-4 text-right">Price</th>
                      <th className="px-6 py-4 text-right">Margin</th>
                      <th className="px-6 py-4">Status</th>
                      <th className="px-6 py-4 text-right">Actions</th>
                    </tr>
                  </thead>
                  <tbody className="divide-y divide-slate-100">
                    {filteredInventory.map((item) => {
                      const total = item.stock.warehouse + item.stock.store;
                      const margin = ((item.price - item.cost) / item.price * 100).toFixed(0);
                      return (
                        <tr key={item.id} className="hover:bg-slate-50/50 transition-colors">
                          <td className="px-6 py-4">
                            <div className="font-medium text-slate-900">{item.name}</div>
                            <div className="text-xs text-slate-400 font-mono mt-0.5">{item.id}</div>
                          </td>
                          <td className="px-6 py-4">
                            <div className="flex flex-col gap-1 text-xs">
                              <span className="flex justify-between w-32"><span className="text-slate-500">Wholesale:</span> <span className="font-mono">{item.stock.warehouse}</span></span>
                              <span className="flex justify-between w-32"><span className="text-slate-500">Store:</span> <span className="font-mono">{item.stock.store}</span></span>
                              <div className="w-32 h-1 bg-slate-100 rounded-full mt-1 overflow-hidden">
                                <div className="h-full bg-slate-900" style={{ width: `${(total / (item.safetyStock * 3)) * 100}%` }}></div>
                              </div>
                            </div>
                          </td>
                          <td className="px-6 py-4 text-right font-mono text-slate-600">${item.cost.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right font-mono text-slate-900">${item.price.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right">
                            <span className={`inline-block px-2 py-0.5 rounded text-xs font-medium ${margin > 50 ? 'bg-emerald-100 text-emerald-700' : 'bg-amber-100 text-amber-700'}`}>
                              {margin}%
                            </span>
                          </td>
                          <td className="px-6 py-4">
                            <Badge status={total < item.safetyStock ? 'Low' : 'Normal'} />
                          </td>
                          <td className="px-6 py-4 text-right text-slate-400">
                            <button className="hover:text-slate-900">Edit</button>
                          </td>
                        </tr>
                      );
                    })}
                  </tbody>
                </table>
              </div>
            </Card>
          </div>
        );

      case 'orders':
        return (
          <div className="space-y-6 animate-fade-in">
             <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">All Orders</h2>
              <div className="text-sm text-slate-500">Syncing with Shopify & Amazon...</div>
            </div>
            <Card className="overflow-hidden">
              <table className="w-full text-left text-sm">
                <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                  <tr>
                    <th className="px-6 py-4">Order ID</th>
                    <th className="px-6 py-4">Customer</th>
                    <th className="px-6 py-4">Channel</th>
                    <th className="px-6 py-4">Date</th>
                    <th className="px-6 py-4 text-right">Total</th>
                    <th className="px-6 py-4">Status</th>
                    <th className="px-6 py-4 text-right">Action</th>
                  </tr>
                </thead>
                <tbody className="divide-y divide-slate-100">
                  {orders.map((order) => (
                    <tr key={order.id} className="hover:bg-slate-50/50">
                      <td className="px-6 py-4 font-mono font-medium text-slate-900">{order.id}</td>
                      <td className="px-6 py-4">{order.customer}</td>
                      <td className="px-6 py-4">
                        <span className="flex items-center gap-2">
                           <span className={`w-2 h-2 rounded-full ${order.channel === 'Shopify' ? 'bg-green-500' : order.channel === 'Amazon' ? 'bg-orange-500' : 'bg-blue-500'}`}></span>
                           {order.channel}
                        </span>
                      </td>
                      <td className="px-6 py-4 text-slate-500">{order.date}</td>
                      <td className="px-6 py-4 text-right font-medium">${order.total.toFixed(2)}</td>
                      <td className="px-6 py-4"><Badge status={order.status} /></td>
                      <td className="px-6 py-4 text-right">
                        {order.status === 'Pending' && (
                          <button
                            onClick={() => fulfillOrder(order.id)}
                            className="text-xs bg-slate-900 text-white px-3 py-1.5 rounded-md hover:bg-slate-700 transition-colors"
                          >
                            Ship
                          </button>
                        )}
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </Card>
          </div>
        );

      case 'purchasing':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">Purchase Orders</h2>
              <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                <Plus className="w-4 h-4 mr-2" /> New PO
              </button>
            </div>
             <div className="grid gap-6">
                {purchaseOrders.map((po) => (
                  <Card key={po.id} className="p-0 overflow-hidden flex flex-col md:flex-row">
                     <div className="p-6 flex-1">
                        <div className="flex justify-between items-start mb-4">
                          <div>
                            <div className="flex items-center gap-3">
                              <h3 className="font-bold text-lg text-slate-900">{po.id}</h3>
                              <Badge status={po.status} />
                            </div>
                            <p className="text-slate-500 text-sm mt-1">Vendor: <span className="font-medium text-slate-700">{po.vendor}</span></p>
                          </div>
                          <div className="text-right">
                             <div className="font-mono font-bold text-lg">${po.total.toFixed(2)}</div>
                             <div className="text-xs text-slate-400">Total Value</div>
                          </div>
                        </div>
                        <div className="flex items-center gap-6 text-sm text-slate-600">
                           <div className="flex items-center gap-2">
                              <Truck className="w-4 h-4" />
                              <span>Expected: {po.expected}</span>
                           </div>
                           <div className="flex items-center gap-2">
                              <DollarSign className="w-4 h-4" />
                              <span>Payment: Net 30</span>
                           </div>
                        </div>
                     </div>
                     {po.status !== 'Received' && (
                       <div className="bg-slate-50 p-6 flex items-center border-t md:border-t-0 md:border-l border-slate-100">
                          <button
                            onClick={() => receivePO(po.id)}
                            className="w-full md:w-auto flex items-center justify-center px-6 py-2 bg-emerald-600 text-white rounded-lg hover:bg-emerald-700 transition-colors font-medium text-sm"
                          >
                            <CheckCircle className="w-4 h-4 mr-2" />
                            Receive Goods
                          </button>
                       </div>
                     )}
                  </Card>
                ))}
             </div>
          </div>
        );

      default:
        return <div className="p-10 text-center text-slate-400">Module under construction</div>;
    }
  };

  return (
    <div className="flex h-screen bg-slate-50 font-sans text-slate-900 overflow-hidden">
      {/* Added style block to define the animation which was missing in Tailwind default config */}
      <style>{`
        @keyframes fadeIn {
          from { opacity: 0; transform: translateY(10px); }
          to { opacity: 1; transform: translateY(0); }
        }
        .animate-fade-in {
          animation: fadeIn 0.4s ease-out forwards;
        }
      `}</style>

      {/* Sidebar for Desktop */}
      <aside className="hidden md:flex flex-col w-64 bg-white border-r border-slate-200 h-full">
        <div className="p-6 flex items-center border-b border-slate-100">
          <div className="w-8 h-8 bg-indigo-600 rounded-lg flex items-center justify-center mr-3 shadow-sm">
            <Package className="w-5 h-5 text-white" />
          </div>
          <span className="font-bold text-lg tracking-tight">ZeroERP</span>
        </div>
        <div className="flex-1 overflow-y-auto py-6 px-4">
          <Navigation />
        </div>
        <div className="p-4 border-t border-slate-100">
          <div className="flex items-center p-2 rounded-lg bg-slate-50 border border-slate-100">
            <div className="w-8 h-8 rounded-full bg-slate-200 flex items-center justify-center text-xs font-bold text-slate-600">
              JS
            </div>
            <div className="ml-3">
              <p className="text-xs font-bold text-slate-900">John Smith</p>
              <p className="text-[10px] text-slate-500">Admin</p>
            </div>
          </div>
        </div>
      </aside>

      {/* Mobile Sidebar Overlay */}
      {isMobileMenuOpen && (
        <div className="fixed inset-0 z-50 bg-slate-900/50 md:hidden" onClick={() => setIsMobileMenuOpen(false)}>
          <div className="absolute left-0 top-0 bottom-0 w-64 bg-white p-4" onClick={e => e.stopPropagation()}>
             <div className="flex justify-between items-center mb-6">
                <span className="font-bold text-lg">ZeroERP</span>
                <button onClick={() => setIsMobileMenuOpen(false)}><X className="w-6 h-6" /></button>
             </div>
             <Navigation />
          </div>
        </div>
      )}

      {/* Main Content Area */}
      <main className="flex-1 flex flex-col h-full overflow-hidden">
        {/* Top Header */}
        <header className="h-16 bg-white border-b border-slate-200 flex justify-between items-center px-6">
          <div className="flex items-center md:hidden">
            <button onClick={() => setIsMobileMenuOpen(true)} className="mr-4">
              <Menu className="w-6 h-6 text-slate-600" />
            </button>
          </div>

          <div className="hidden md:flex flex-1 max-w-md relative">
            <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
            <input
              type="text"
              placeholder="Command + K to search..."
              className="w-full pl-10 pr-4 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-indigo-500/20 focus:border-indigo-500 transition-all"
            />
            <div className="absolute right-3 top-1/2 -translate-y-1/2 flex gap-1">
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">Cmd</span>
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">K</span>
            </div>
          </div>

          <div className="flex items-center gap-4">
            <button className="relative p-2 text-slate-400 hover:bg-slate-50 rounded-lg transition-colors">
              <Bell className="w-5 h-5" />
              <span className="absolute top-2 right-2 w-2 h-2 bg-rose-500 rounded-full border-2 border-white"></span>
            </button>
          </div>
        </header>

        {/* Scrollable Page Content */}
        <div className="flex-1 overflow-auto p-4 md:p-8">
           <div className="max-w-7xl mx-auto">
             {renderContent()}
           </div>
        </div>
      </main>
    </div>
  );
}

Overwriting ZeroERP.js


### How to Run ZeroERP Locally

This application uses **React** and **Lucide React** for icons. To run it, you'll need [Node.js](https://nodejs.org/) installed.

1.  **Create a new project** using Vite (a fast build tool):
    ```bash
    npm create vite@latest zero-erp -- --template react
    cd zero-erp
    ```

2.  **Install the icon library**:
    ```bash
    npm install lucide-react
    ```

3.  **Setup the file**:
    *   Rename the downloaded `ZeroERP.js` to `App.jsx`.
    *   Replace the existing `src/App.jsx` in your project folder with this file.
    *   *Note: Ensure `src/main.jsx` (or `index.js`) imports Tailwind CSS if you want the styling to work exactly as shown, or add the Tailwind CDN to your `index.html` head:*
    ```html
    <script src="https://cdn.tailwindcss.com"></script>
    ```

4.  **Run the application**:
    ```bash
    npm run dev
    ```
    Open the link shown in your terminal (usually `http://localhost:5173`) to view the app.

In [4]:
from google.colab import files

# Download the file to your local machine
try:
    files.download('ZeroERP.js')
except ImportError:
    pass # Handle cases where files.download is not available (e.g. non-Colab env)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Modify `ZeroERP.js` to implement the CSV export functionality for the inventory. Create a `handleExport` function that converts the `inventory` state into a CSV string (including headers: ID, Name, Category, Warehouse Stock, Store Stock, Safety Stock, Cost, Price, Vendor), creates a blob, and triggers a download named 'inventory_export.csv'. Attach this function to the `onClick` event of the 'Export' button in the Inventory section.

## Implement Export Functionality

### Subtask:
Add logic to the 'Export' button to generate and download a CSV file of the current inventory.


**Reasoning**:
I will update the `ZeroERP.js` file to implement the export functionality. This involves defining the `handleExport` function within the `App` component to generate a CSV from the inventory state and triggering a download. I will also attach this function to the 'Export' button's `onClick` event.



In [1]:
%%writefile ZeroERP.js
import React, { useState, useMemo } from 'react';
import {
  LayoutDashboard,
  Package,
  ShoppingCart,
  Truck,
  Settings,
  Search,
  Bell,
  Menu,
  X,
  ArrowUpRight,
  ArrowDownRight,
  Plus,
  Filter,
  Download,
  CheckCircle,
  AlertCircle,
  DollarSign,
  BarChart3
} from 'lucide-react';

// --- Mock Data Generators ---

const INITIAL_INVENTORY = [
  { id: 'SKU-001', name: 'Ceramic Pour-Over Set', category: 'Coffee', stock: { warehouse: 45, store: 12 }, safetyStock: 20, cost: 15.00, price: 45.00, vendor: 'CeramicsCo' },
  { id: 'SKU-002', name: 'Ethiopian Yirgacheffe (12oz)', category: 'Coffee', stock: { warehouse: 120, store: 50 }, safetyStock: 50, cost: 8.50, price: 22.00, vendor: 'GlobalBeans' },
  { id: 'SKU-003', name: 'Gooseneck Kettle (Matte Black)', category: 'Equipment', stock: { warehouse: 5, store: 2 }, safetyStock: 10, cost: 35.00, price: 85.00, vendor: 'HeatSync' },
  { id: 'SKU-004', name: 'Paper Filters (100ct)', category: 'Consumables', stock: { warehouse: 500, store: 100 }, safetyStock: 200, cost: 2.00, price: 8.00, vendor: 'PaperWorks' },
  { id: 'SKU-005', name: 'Barista Apron (Canvas)', category: 'Apparel', stock: { warehouse: 25, store: 5 }, safetyStock: 15, cost: 12.00, price: 40.00, vendor: 'TextileBros' },
  { id: 'SKU-006', name: 'Cold Brew Kit', category: 'Bundles', stock: { warehouse: 15, store: 0 }, safetyStock: 10, cost: 18.00, price: 55.00, vendor: 'Internal' },
];

const INITIAL_ORDERS = [
  { id: 'ORD-7782', customer: 'Alice Freeman', channel: 'Shopify', total: 135.00, status: 'Pending', date: '2023-10-24', items: 3 },
  { id: 'ORD-7781', customer: 'Mark Stone', channel: 'Amazon', total: 22.00, status: 'Shipped', date: '2023-10-24', items: 1 },
  { id: 'ORD-7780', customer: 'Cafe Lux (Wholesale)', channel: 'B2B Portal', total: 450.00, status: 'Shipped', date: '2023-10-23', items: 20 },
  { id: 'ORD-7779', customer: 'Sarah Connor', channel: 'Shopify', total: 85.00, status: 'Delivered', date: '2023-10-23', items: 1 },
];

const INITIAL_PO = [
  { id: 'PO-9001', vendor: 'HeatSync', status: 'Ordered', expected: '2023-11-01', total: 1500.00 },
  { id: 'PO-9002', vendor: 'GlobalBeans', status: 'Received', expected: '2023-10-20', total: 850.00 },
];

// --- Components ---

const Card = ({ children, className = "" }) => (
  <div className={`bg-white rounded-xl border border-slate-200 shadow-sm ${className}`}>
    {children}
  </div>
);

const Badge = ({ status }) => {
  const styles = {
    Pending: "bg-amber-100 text-amber-700",
    Shipped: "bg-blue-100 text-blue-700",
    Delivered: "bg-emerald-100 text-emerald-700",
    Ordered: "bg-purple-100 text-purple-700",
    Received: "bg-slate-100 text-slate-700",
    Low: "bg-red-100 text-red-700",
    Normal: "bg-emerald-100 text-emerald-700",
  };
  return (
    <span className={`px-2.5 py-0.5 rounded-full text-xs font-medium ${styles[status] || "bg-gray-100 text-gray-800"}`}>
      {status}
    </span>
  );
};

const MetricCard = ({ title, value, subtext, trend, trendUp }) => (
  <Card className="p-5">
    <div className="flex justify-between items-start">
      <div>
        <p className="text-sm font-medium text-slate-500">{title}</p>
        <h3 className="text-2xl font-bold text-slate-900 mt-1">{value}</h3>
      </div>
      <div className={`p-2 rounded-lg ${trendUp ? 'bg-emerald-50' : 'bg-rose-50'}`}>
        {trendUp ? <ArrowUpRight className="w-5 h-5 text-emerald-600" /> : <ArrowDownRight className="w-5 h-5 text-rose-600" />}
      </div>
    </div>
    <div className="mt-4 flex items-center text-xs">
      <span className={`font-medium ${trendUp ? 'text-emerald-600' : 'text-rose-600'}`}>
        {trend}
      </span>
      <span className="text-slate-400 ml-1">{subtext}</span>
    </div>
  </Card>
);

// --- Main Application ---

export default function App() {
  const [activeTab, setActiveTab] = useState('dashboard');
  const [isMobileMenuOpen, setIsMobileMenuOpen] = useState(false);

  // App State
  const [inventory, setInventory] = useState(INITIAL_INVENTORY);
  const [orders, setOrders] = useState(INITIAL_ORDERS);
  const [purchaseOrders, setPurchaseOrders] = useState(INITIAL_PO);
  const [searchTerm, setSearchTerm] = useState('');

  // Derived Metrics
  const totalStockValue = useMemo(() =>
    inventory.reduce((acc, item) => acc + ((item.stock.warehouse + item.stock.store) * item.cost), 0),
  [inventory]);

  const lowStockCount = useMemo(() =>
    inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).length,
  [inventory]);

  const pendingOrdersCount = useMemo(() =>
    orders.filter(o => o.status === 'Pending').length,
  [orders]);

  // Actions
  const fulfillOrder = (orderId) => {
    setOrders(orders.map(o => o.id === orderId ? { ...o, status: 'Shipped' } : o));
    // In a real app, this would decrement inventory
    alert(`Order ${orderId} marked as Shipped. (Simulation: Inventory would be adjusted here)`);
  };

  const receivePO = (poId) => {
    setPurchaseOrders(purchaseOrders.map(p => p.id === poId ? { ...p, status: 'Received' } : p));
    alert(`PO ${poId} received. (Simulation: Stock levels would be updated here)`);
  };

  // Export Functionality
  const handleExport = () => {
    // Define headers
    const headers = ['ID,Name,Category,Warehouse Stock,Store Stock,Safety Stock,Cost,Price,Vendor'];

    // Map inventory to CSV rows
    const rows = inventory.map(item => {
      return [
        item.id,
        `"${item.name}"`, // Quote name to handle potential commas
        item.category,
        item.stock.warehouse,
        item.stock.store,
        item.safetyStock,
        item.cost.toFixed(2),
        item.price.toFixed(2),
        item.vendor
      ].join(',');
    });

    // Combine headers and rows
    const csvString = [headers, ...rows].join('\n');

    // Create Blob and download link
    const blob = new Blob([csvString], { type: 'text/csv' });
    const url = URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = 'inventory_export.csv';
    document.body.appendChild(a);
    a.click();

    // Cleanup
    document.body.removeChild(a);
    URL.revokeObjectURL(url);
  };

  const filteredInventory = inventory.filter(item =>
    item.name.toLowerCase().includes(searchTerm.toLowerCase()) ||
    item.id.toLowerCase().includes(searchTerm.toLowerCase())
  );

  const Navigation = () => (
    <nav className="space-y-1">
      {[
        { id: 'dashboard', label: 'Dashboard', icon: LayoutDashboard },
        { id: 'inventory', label: 'Inventory', icon: Package },
        { id: 'orders', label: 'Orders', icon: ShoppingCart },
        { id: 'purchasing', label: 'Purchasing', icon: Truck },
        { id: 'reports', label: 'Reports', icon: BarChart3 },
        { id: 'settings', label: 'Settings', icon: Settings },
      ].map((item) => (
        <button
          key={item.id}
          onClick={() => {
            setActiveTab(item.id);
            setIsMobileMenuOpen(false);
          }}
          className={`w-full flex items-center px-4 py-3 text-sm font-medium rounded-lg transition-colors ${
            activeTab === item.id
              ? 'bg-slate-900 text-white'
              : 'text-slate-600 hover:bg-slate-100'
          }`}
        >
          <item.icon className="w-5 h-5 mr-3" />
          {item.label}
          {item.id === 'orders' && pendingOrdersCount > 0 && (
            <span className="ml-auto bg-rose-500 text-white text-xs font-bold px-2 py-0.5 rounded-full">
              {pendingOrdersCount}
            </span>
          )}
        </button>
      ))}
    </nav>
  );

  const renderContent = () => {
    switch(activeTab) {
      case 'dashboard':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4">
              <MetricCard
                title="Total Stock Value"
                value={`$${totalStockValue.toLocaleString()}`}
                trend="+12%"
                subtext="vs last month"
                trendUp={true}
              />
              <MetricCard
                title="Pending Orders"
                value={pendingOrdersCount}
                trend="-5%"
                subtext="processing faster"
                trendUp={true}
              />
              <MetricCard
                title="Low Stock SKUs"
                value={lowStockCount}
                trend="+2"
                subtext="needs attention"
                trendUp={false}
              />
              <MetricCard
                title="Gross Margin (Avg)"
                value="68%"
                trend="+1.2%"
                subtext="healthy"
                trendUp={true}
              />
            </div>

            <div className="grid grid-cols-1 lg:grid-cols-3 gap-6">
              <Card className="col-span-2 p-6">
                <div className="flex justify-between items-center mb-6">
                  <h3 className="font-bold text-slate-800">Recent Sales Velocity</h3>
                  <select className="text-sm border-slate-200 rounded-md text-slate-500">
                    <option>Last 7 Days</option>
                  </select>
                </div>
                {/* Simulated Chart */}
                <div className="h-64 flex items-end space-x-2">
                  {[45, 60, 35, 70, 85, 60, 75].map((h, i) => (
                    <div key={i} className="flex-1 bg-slate-100 rounded-t-lg relative group">
                      <div
                        className="absolute bottom-0 w-full bg-slate-800 rounded-t-lg transition-all duration-500 hover:bg-indigo-600"
                        style={{ height: `${h}%` }}
                      ></div>
                      <div className="opacity-0 group-hover:opacity-100 absolute -top-8 left-1/2 -translate-x-1/2 bg-slate-900 text-white text-xs py-1 px-2 rounded">
                        ${h * 100}
                      </div>
                    </div>
                  ))}
                </div>
                <div className="flex justify-between mt-2 text-xs text-slate-400">
                  <span>Mon</span><span>Tue</span><span>Wed</span><span>Thu</span><span>Fri</span><span>Sat</span><span>Sun</span>
                </div>
              </Card>

              <Card className="p-6">
                <h3 className="font-bold text-slate-800 mb-4">Stock Alerts</h3>
                <div className="space-y-4">
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).map(item => (
                    <div key={item.id} className="flex items-start p-3 bg-red-50 rounded-lg border border-red-100">
                      <AlertCircle className="w-5 h-5 text-red-500 mt-0.5 mr-3 flex-shrink-0" />
                      <div>
                        <p className="text-sm font-medium text-red-900">{item.name}</p>
                        <p className="text-xs text-red-700 mt-1">
                          Only {item.stock.warehouse + item.stock.store} units left (Safety: {item.safetyStock})
                        </p>
                        <button className="mt-2 text-xs font-medium text-red-600 hover:text-red-800">
                          Create PO &rarr;
                        </button>
                      </div>
                    </div>
                  ))}
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) >= i.safetyStock).slice(0, 2).map(item => (
                    <div key={item.id} className="flex items-center p-3 hover:bg-slate-50 rounded-lg transition-colors">
                      <div className="w-2 h-2 rounded-full bg-emerald-500 mr-3"></div>
                      <div className="flex-1">
                        <p className="text-sm font-medium text-slate-700">{item.name}</p>
                        <p className="text-xs text-slate-500">{item.stock.warehouse + item.stock.store} Units Available</p>
                      </div>
                    </div>
                  ))}
                </div>
              </Card>
            </div>
          </div>
        );

      case 'inventory':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex flex-col sm:flex-row justify-between items-start sm:items-center gap-4">
              <h2 className="text-xl font-bold text-slate-900">Inventory Management</h2>
              <div className="flex gap-2">
                <button
                  onClick={handleExport}
                  className="flex items-center px-4 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50"
                >
                  <Download className="w-4 h-4 mr-2" /> Export
                </button>
                <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                  <Plus className="w-4 h-4 mr-2" /> Add SKU
                </button>
              </div>
            </div>

            <Card className="overflow-hidden">
              <div className="p-4 border-b border-slate-100 bg-slate-50/50 flex flex-col sm:flex-row gap-4">
                <div className="relative flex-1">
                  <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
                  <input
                    type="text"
                    placeholder="Search SKUs, names, or categories..."
                    className="w-full pl-10 pr-4 py-2 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
                    value={searchTerm}
                    onChange={(e) => setSearchTerm(e.target.value)}
                  />
                </div>
                <button className="flex items-center px-3 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50">
                  <Filter className="w-4 h-4 mr-2" /> Filters
                </button>
              </div>

              <div className="overflow-x-auto">
                <table className="w-full text-left text-sm">
                  <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                    <tr>
                      <th className="px-6 py-4">Product / SKU</th>
                      <th className="px-6 py-4">Location Split</th>
                      <th className="px-6 py-4 text-right">Cost</th>
                      <th className="px-6 py-4 text-right">Price</th>
                      <th className="px-6 py-4 text-right">Margin</th>
                      <th className="px-6 py-4">Status</th>
                      <th className="px-6 py-4 text-right">Actions</th>
                    </tr>
                  </thead>
                  <tbody className="divide-y divide-slate-100">
                    {filteredInventory.map((item) => {
                      const total = item.stock.warehouse + item.stock.store;
                      const margin = ((item.price - item.cost) / item.price * 100).toFixed(0);
                      return (
                        <tr key={item.id} className="hover:bg-slate-50/50 transition-colors">
                          <td className="px-6 py-4">
                            <div className="font-medium text-slate-900">{item.name}</div>
                            <div className="text-xs text-slate-400 font-mono mt-0.5">{item.id}</div>
                          </td>
                          <td className="px-6 py-4">
                            <div className="flex flex-col gap-1 text-xs">
                              <span className="flex justify-between w-32"><span className="text-slate-500">Wholesale:</span> <span className="font-mono">{item.stock.warehouse}</span></span>
                              <span className="flex justify-between w-32"><span className="text-slate-500">Store:</span> <span className="font-mono">{item.stock.store}</span></span>
                              <div className="w-32 h-1 bg-slate-100 rounded-full mt-1 overflow-hidden">
                                <div className="h-full bg-slate-900" style={{ width: `${(total / (item.safetyStock * 3)) * 100}%` }}></div>
                              </div>
                            </div>
                          </td>
                          <td className="px-6 py-4 text-right font-mono text-slate-600">${item.cost.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right font-mono text-slate-900">${item.price.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right">
                            <span className={`inline-block px-2 py-0.5 rounded text-xs font-medium ${margin > 50 ? 'bg-emerald-100 text-emerald-700' : 'bg-amber-100 text-amber-700'}`}>
                              {margin}%
                            </span>
                          </td>
                          <td className="px-6 py-4">
                            <Badge status={total < item.safetyStock ? 'Low' : 'Normal'} />
                          </td>
                          <td className="px-6 py-4 text-right text-slate-400">
                            <button className="hover:text-slate-900">Edit</button>
                          </td>
                        </tr>
                      );
                    })}
                  </tbody>
                </table>
              </div>
            </Card>
          </div>
        );

      case 'orders':
        return (
          <div className="space-y-6 animate-fade-in">
             <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">All Orders</h2>
              <div className="text-sm text-slate-500">Syncing with Shopify & Amazon...</div>
            </div>
            <Card className="overflow-hidden">
              <table className="w-full text-left text-sm">
                <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                  <tr>
                    <th className="px-6 py-4">Order ID</th>
                    <th className="px-6 py-4">Customer</th>
                    <th className="px-6 py-4">Channel</th>
                    <th className="px-6 py-4">Date</th>
                    <th className="px-6 py-4 text-right">Total</th>
                    <th className="px-6 py-4">Status</th>
                    <th className="px-6 py-4 text-right">Action</th>
                  </tr>
                </thead>
                <tbody className="divide-y divide-slate-100">
                  {orders.map((order) => (
                    <tr key={order.id} className="hover:bg-slate-50/50">
                      <td className="px-6 py-4 font-mono font-medium text-slate-900">{order.id}</td>
                      <td className="px-6 py-4">{order.customer}</td>
                      <td className="px-6 py-4">
                        <span className="flex items-center gap-2">
                           <span className={`w-2 h-2 rounded-full ${order.channel === 'Shopify' ? 'bg-green-500' : order.channel === 'Amazon' ? 'bg-orange-500' : 'bg-blue-500'}`}></span>
                           {order.channel}
                        </span>
                      </td>
                      <td className="px-6 py-4 text-slate-500">{order.date}</td>
                      <td className="px-6 py-4 text-right font-medium">${order.total.toFixed(2)}</td>
                      <td className="px-6 py-4"><Badge status={order.status} /></td>
                      <td className="px-6 py-4 text-right">
                        {order.status === 'Pending' && (
                          <button
                            onClick={() => fulfillOrder(order.id)}
                            className="text-xs bg-slate-900 text-white px-3 py-1.5 rounded-md hover:bg-slate-700 transition-colors"
                          >
                            Ship
                          </button>
                        )}
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </Card>
          </div>
        );

      case 'purchasing':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">Purchase Orders</h2>
              <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                <Plus className="w-4 h-4 mr-2" /> New PO
              </button>
            </div>
             <div className="grid gap-6">
                {purchaseOrders.map((po) => (
                  <Card key={po.id} className="p-0 overflow-hidden flex flex-col md:flex-row">
                     <div className="p-6 flex-1">
                        <div className="flex justify-between items-start mb-4">
                          <div>
                            <div className="flex items-center gap-3">
                              <h3 className="font-bold text-lg text-slate-900">{po.id}</h3>
                              <Badge status={po.status} />
                            </div>
                            <p className="text-slate-500 text-sm mt-1">Vendor: <span className="font-medium text-slate-700">{po.vendor}</span></p>
                          </div>
                          <div className="text-right">
                             <div className="font-mono font-bold text-lg">${po.total.toFixed(2)}</div>
                             <div className="text-xs text-slate-400">Total Value</div>
                          </div>
                        </div>
                        <div className="flex items-center gap-6 text-sm text-slate-600">
                           <div className="flex items-center gap-2">
                              <Truck className="w-4 h-4" />
                              <span>Expected: {po.expected}</span>
                           </div>
                           <div className="flex items-center gap-2">
                              <DollarSign className="w-4 h-4" />
                              <span>Payment: Net 30</span>
                           </div>
                        </div>
                     </div>
                     {po.status !== 'Received' && (
                       <div className="bg-slate-50 p-6 flex items-center border-t md:border-t-0 md:border-l border-slate-100">
                          <button
                            onClick={() => receivePO(po.id)}
                            className="w-full md:w-auto flex items-center justify-center px-6 py-2 bg-emerald-600 text-white rounded-lg hover:bg-emerald-700 transition-colors font-medium text-sm"
                          >
                            <CheckCircle className="w-4 h-4 mr-2" />
                            Receive Goods
                          </button>
                       </div>
                     )}
                  </Card>
                ))}
             </div>
          </div>
        );

      default:
        return <div className="p-10 text-center text-slate-400">Module under construction</div>;
    }
  };

  return (
    <div className="flex h-screen bg-slate-50 font-sans text-slate-900 overflow-hidden">
      {/* Added style block to define the animation which was missing in Tailwind default config */}
      <style>{`
        @keyframes fadeIn {
          from { opacity: 0; transform: translateY(10px); }
          to { opacity: 1; transform: translateY(0); }
        }
        .animate-fade-in {
          animation: fadeIn 0.4s ease-out forwards;
        }
      `}</style>

      {/* Sidebar for Desktop */}
      <aside className="hidden md:flex flex-col w-64 bg-white border-r border-slate-200 h-full">
        <div className="p-6 flex items-center border-b border-slate-100">
          <div className="w-8 h-8 bg-indigo-600 rounded-lg flex items-center justify-center mr-3 shadow-sm">
            <Package className="w-5 h-5 text-white" />
          </div>
          <span className="font-bold text-lg tracking-tight">ZeroERP</span>
        </div>
        <div className="flex-1 overflow-y-auto py-6 px-4">
          <Navigation />
        </div>
        <div className="p-4 border-t border-slate-100">
          <div className="flex items-center p-2 rounded-lg bg-slate-50 border border-slate-100">
            <div className="w-8 h-8 rounded-full bg-slate-200 flex items-center justify-center text-xs font-bold text-slate-600">
              JS
            </div>
            <div className="ml-3">
              <p className="text-xs font-bold text-slate-900">John Smith</p>
              <p className="text-[10px] text-slate-500">Admin</p>
            </div>
          </div>
        </div>
      </aside>

      {/* Mobile Sidebar Overlay */}
      {isMobileMenuOpen && (
        <div className="fixed inset-0 z-50 bg-slate-900/50 md:hidden" onClick={() => setIsMobileMenuOpen(false)}>
          <div className="absolute left-0 top-0 bottom-0 w-64 bg-white p-4" onClick={e => e.stopPropagation()}>
             <div className="flex justify-between items-center mb-6">
                <span className="font-bold text-lg">ZeroERP</span>
                <button onClick={() => setIsMobileMenuOpen(false)}><X className="w-6 h-6" /></button>
             </div>
             <Navigation />
          </div>
        </div>
      )}

      {/* Main Content Area */}
      <main className="flex-1 flex flex-col h-full overflow-hidden">
        {/* Top Header */}
        <header className="h-16 bg-white border-b border-slate-200 flex justify-between items-center px-6">
          <div className="flex items-center md:hidden">
            <button onClick={() => setIsMobileMenuOpen(true)} className="mr-4">
              <Menu className="w-6 h-6 text-slate-600" />
            </button>
          </div>

          <div className="hidden md:flex flex-1 max-w-md relative">
            <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
            <input
              type="text"
              placeholder="Command + K to search..."
              className="w-full pl-10 pr-4 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-indigo-500/20 focus:border-indigo-500 transition-all"
            />
            <div className="absolute right-3 top-1/2 -translate-y-1/2 flex gap-1">
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">Cmd</span>
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">K</span>
            </div>
          </div>

          <div className="flex items-center gap-4">
            <button className="relative p-2 text-slate-400 hover:bg-slate-50 rounded-lg transition-colors">
              <Bell className="w-5 h-5" />
              <span className="absolute top-2 right-2 w-2 h-2 bg-rose-500 rounded-full border-2 border-white"></span>
            </button>
          </div>
        </header>

        {/* Scrollable Page Content */}
        <div className="flex-1 overflow-auto p-4 md:p-8">
           <div className="max-w-7xl mx-auto">
             {renderContent()}
           </div>
        </div>
      </main>
    </div>
  );
}

Overwriting ZeroERP.js


## Create Inventory Modals

### Subtask:
Develop the React components for the 'Add SKU' and 'Edit SKU' modals, including the necessary form fields and visibility state management.


**Reasoning**:
Update ZeroERP.js to include Modal and InventoryForm components, and integrate them into the App component with state management for adding and editing items.



In [2]:
%%writefile ZeroERP.js
import React, { useState, useMemo } from 'react';
import {
  LayoutDashboard,
  Package,
  ShoppingCart,
  Truck,
  Settings,
  Search,
  Bell,
  Menu,
  X,
  ArrowUpRight,
  ArrowDownRight,
  Plus,
  Filter,
  Download,
  CheckCircle,
  AlertCircle,
  DollarSign,
  BarChart3
} from 'lucide-react';

// --- Mock Data Generators ---

const INITIAL_INVENTORY = [
  { id: 'SKU-001', name: 'Ceramic Pour-Over Set', category: 'Coffee', stock: { warehouse: 45, store: 12 }, safetyStock: 20, cost: 15.00, price: 45.00, vendor: 'CeramicsCo' },
  { id: 'SKU-002', name: 'Ethiopian Yirgacheffe (12oz)', category: 'Coffee', stock: { warehouse: 120, store: 50 }, safetyStock: 50, cost: 8.50, price: 22.00, vendor: 'GlobalBeans' },
  { id: 'SKU-003', name: 'Gooseneck Kettle (Matte Black)', category: 'Equipment', stock: { warehouse: 5, store: 2 }, safetyStock: 10, cost: 35.00, price: 85.00, vendor: 'HeatSync' },
  { id: 'SKU-004', name: 'Paper Filters (100ct)', category: 'Consumables', stock: { warehouse: 500, store: 100 }, safetyStock: 200, cost: 2.00, price: 8.00, vendor: 'PaperWorks' },
  { id: 'SKU-005', name: 'Barista Apron (Canvas)', category: 'Apparel', stock: { warehouse: 25, store: 5 }, safetyStock: 15, cost: 12.00, price: 40.00, vendor: 'TextileBros' },
  { id: 'SKU-006', name: 'Cold Brew Kit', category: 'Bundles', stock: { warehouse: 15, store: 0 }, safetyStock: 10, cost: 18.00, price: 55.00, vendor: 'Internal' },
];

const INITIAL_ORDERS = [
  { id: 'ORD-7782', customer: 'Alice Freeman', channel: 'Shopify', total: 135.00, status: 'Pending', date: '2023-10-24', items: 3 },
  { id: 'ORD-7781', customer: 'Mark Stone', channel: 'Amazon', total: 22.00, status: 'Shipped', date: '2023-10-24', items: 1 },
  { id: 'ORD-7780', customer: 'Cafe Lux (Wholesale)', channel: 'B2B Portal', total: 450.00, status: 'Shipped', date: '2023-10-23', items: 20 },
  { id: 'ORD-7779', customer: 'Sarah Connor', channel: 'Shopify', total: 85.00, status: 'Delivered', date: '2023-10-23', items: 1 },
];

const INITIAL_PO = [
  { id: 'PO-9001', vendor: 'HeatSync', status: 'Ordered', expected: '2023-11-01', total: 1500.00 },
  { id: 'PO-9002', vendor: 'GlobalBeans', status: 'Received', expected: '2023-10-20', total: 850.00 },
];

// --- Components ---

const Card = ({ children, className = "" }) => (
  <div className={`bg-white rounded-xl border border-slate-200 shadow-sm ${className}`}>
    {children}
  </div>
);

const Badge = ({ status }) => {
  const styles = {
    Pending: "bg-amber-100 text-amber-700",
    Shipped: "bg-blue-100 text-blue-700",
    Delivered: "bg-emerald-100 text-emerald-700",
    Ordered: "bg-purple-100 text-purple-700",
    Received: "bg-slate-100 text-slate-700",
    Low: "bg-red-100 text-red-700",
    Normal: "bg-emerald-100 text-emerald-700",
  };
  return (
    <span className={`px-2.5 py-0.5 rounded-full text-xs font-medium ${styles[status] || "bg-gray-100 text-gray-800"}`}>
      {status}
    </span>
  );
};

const MetricCard = ({ title, value, subtext, trend, trendUp }) => (
  <Card className="p-5">
    <div className="flex justify-between items-start">
      <div>
        <p className="text-sm font-medium text-slate-500">{title}</p>
        <h3 className="text-2xl font-bold text-slate-900 mt-1">{value}</h3>
      </div>
      <div className={`p-2 rounded-lg ${trendUp ? 'bg-emerald-50' : 'bg-rose-50'}`}>
        {trendUp ? <ArrowUpRight className="w-5 h-5 text-emerald-600" /> : <ArrowDownRight className="w-5 h-5 text-rose-600" />}
      </div>
    </div>
    <div className="mt-4 flex items-center text-xs">
      <span className={`font-medium ${trendUp ? 'text-emerald-600' : 'text-rose-600'}`}>
        {trend}
      </span>
      <span className="text-slate-400 ml-1">{subtext}</span>
    </div>
  </Card>
);

// --- Modal Component ---
const Modal = ({ isOpen, onClose, title, children }) => {
  if (!isOpen) return null;
  return (
    <div className="fixed inset-0 z-50 flex items-center justify-center bg-slate-900/50 backdrop-blur-sm p-4 animate-fade-in">
      <div className="bg-white rounded-xl shadow-xl w-full max-w-lg max-h-[90vh] overflow-y-auto">
        <div className="flex justify-between items-center p-4 border-b border-slate-100 sticky top-0 bg-white z-10">
          <h3 className="font-bold text-lg text-slate-800">{title}</h3>
          <button onClick={onClose} className="text-slate-400 hover:text-slate-600">
            <X className="w-5 h-5" />
          </button>
        </div>
        <div className="p-6">
          {children}
        </div>
      </div>
    </div>
  );
};

// --- Inventory Form Component ---
const InventoryForm = ({ initialData = {}, onSubmit, onCancel }) => {
  const [formData, setFormData] = useState({
    name: initialData.name || '',
    category: initialData.category || '',
    warehouse: initialData.stock?.warehouse || 0,
    store: initialData.stock?.store || 0,
    safetyStock: initialData.safetyStock || 0,
    cost: initialData.cost || 0,
    price: initialData.price || 0,
    vendor: initialData.vendor || ''
  });

  const handleChange = (e) => {
    const { name, value } = e.target;
    setFormData(prev => ({
      ...prev,
      [name]: ['warehouse', 'store', 'safetyStock', 'cost', 'price'].includes(name) ? parseFloat(value) || 0 : value
    }));
  };

  const handleSubmit = (e) => {
    e.preventDefault();
    onSubmit(formData);
  };

  const InputField = ({ label, name, type = "text", step }) => (
    <div className="space-y-1">
      <label className="text-xs font-medium text-slate-500 uppercase tracking-wider">{label}</label>
      <input
        type={type}
        name={name}
        step={step}
        value={formData[name]}
        onChange={handleChange}
        className="w-full px-3 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
        required
      />
    </div>
  );

  return (
    <form onSubmit={handleSubmit} className="space-y-4">
      <InputField label="Product Name" name="name" />
      <div className="grid grid-cols-2 gap-4">
        <InputField label="Category" name="category" />
        <InputField label="Vendor" name="vendor" />
      </div>
      <div className="grid grid-cols-2 gap-4">
        <InputField label="Cost ($)" name="cost" type="number" step="0.01" />
        <InputField label="Price ($)" name="price" type="number" step="0.01" />
      </div>
      <div className="p-4 bg-slate-50 rounded-lg space-y-4 border border-slate-100">
        <h4 className="text-sm font-bold text-slate-700">Stock Levels</h4>
        <div className="grid grid-cols-3 gap-3">
          <InputField label="Warehouse" name="warehouse" type="number" />
          <InputField label="Store" name="store" type="number" />
          <InputField label="Safety Stock" name="safetyStock" type="number" />
        </div>
      </div>

      <div className="flex justify-end gap-3 pt-4 border-t border-slate-100">
        <button type="button" onClick={onCancel} className="px-4 py-2 text-sm font-medium text-slate-600 bg-white border border-slate-200 rounded-lg hover:bg-slate-50">Cancel</button>
        <button type="submit" className="px-4 py-2 text-sm font-medium text-white bg-slate-900 rounded-lg hover:bg-slate-800">Save Changes</button>
      </div>
    </form>
  );
};

// --- Main Application ---

export default function App() {
  const [activeTab, setActiveTab] = useState('dashboard');
  const [isMobileMenuOpen, setIsMobileMenuOpen] = useState(false);

  // App State
  const [inventory, setInventory] = useState(INITIAL_INVENTORY);
  const [orders, setOrders] = useState(INITIAL_ORDERS);
  const [purchaseOrders, setPurchaseOrders] = useState(INITIAL_PO);
  const [searchTerm, setSearchTerm] = useState('');

  // Modal State
  const [isAddModalOpen, setIsAddModalOpen] = useState(false);
  const [editingItem, setEditingItem] = useState(null);

  // Derived Metrics
  const totalStockValue = useMemo(() =>
    inventory.reduce((acc, item) => acc + ((item.stock.warehouse + item.stock.store) * item.cost), 0),
  [inventory]);

  const lowStockCount = useMemo(() =>
    inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).length,
  [inventory]);

  const pendingOrdersCount = useMemo(() =>
    orders.filter(o => o.status === 'Pending').length,
  [orders]);

  // Actions
  const fulfillOrder = (orderId) => {
    setOrders(orders.map(o => o.id === orderId ? { ...o, status: 'Shipped' } : o));
    alert(`Order ${orderId} marked as Shipped. (Simulation: Inventory would be adjusted here)`);
  };

  const receivePO = (poId) => {
    setPurchaseOrders(purchaseOrders.map(p => p.id === poId ? { ...p, status: 'Received' } : p));
    alert(`PO ${poId} received. (Simulation: Stock levels would be updated here)`);
  };

  const handleExport = () => {
    const headers = ['ID,Name,Category,Warehouse Stock,Store Stock,Safety Stock,Cost,Price,Vendor'];
    const rows = inventory.map(item => {
      return [
        item.id,
        `"${item.name}"`,
        item.category,
        item.stock.warehouse,
        item.stock.store,
        item.safetyStock,
        item.cost.toFixed(2),
        item.price.toFixed(2),
        item.vendor
      ].join(',');
    });
    const csvString = [headers, ...rows].join('\n');
    const blob = new Blob([csvString], { type: 'text/csv' });
    const url = URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = 'inventory_export.csv';
    document.body.appendChild(a);
    a.click();
    document.body.removeChild(a);
    URL.revokeObjectURL(url);
  };

  // Placeholder for modal submit
  const handleInventorySubmit = (data) => {
    // This will be implemented fully in the next step
    console.log("Form Submitted:", data);
    alert("Item saved! (State update logic coming soon)");
    setIsAddModalOpen(false);
    setEditingItem(null);
  };

  const filteredInventory = inventory.filter(item =>
    item.name.toLowerCase().includes(searchTerm.toLowerCase()) ||
    item.id.toLowerCase().includes(searchTerm.toLowerCase())
  );

  const Navigation = () => (
    <nav className="space-y-1">
      {[
        { id: 'dashboard', label: 'Dashboard', icon: LayoutDashboard },
        { id: 'inventory', label: 'Inventory', icon: Package },
        { id: 'orders', label: 'Orders', icon: ShoppingCart },
        { id: 'purchasing', label: 'Purchasing', icon: Truck },
        { id: 'reports', label: 'Reports', icon: BarChart3 },
        { id: 'settings', label: 'Settings', icon: Settings },
      ].map((item) => (
        <button
          key={item.id}
          onClick={() => {
            setActiveTab(item.id);
            setIsMobileMenuOpen(false);
          }}
          className={`w-full flex items-center px-4 py-3 text-sm font-medium rounded-lg transition-colors ${
            activeTab === item.id
              ? 'bg-slate-900 text-white'
              : 'text-slate-600 hover:bg-slate-100'
          }`}
        >
          <item.icon className="w-5 h-5 mr-3" />
          {item.label}
          {item.id === 'orders' && pendingOrdersCount > 0 && (
            <span className="ml-auto bg-rose-500 text-white text-xs font-bold px-2 py-0.5 rounded-full">
              {pendingOrdersCount}
            </span>
          )}
        </button>
      ))}
    </nav>
  );

  const renderContent = () => {
    switch(activeTab) {
      case 'dashboard':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4">
              <MetricCard
                title="Total Stock Value"
                value={`$${totalStockValue.toLocaleString()}`}
                trend="+12%"
                subtext="vs last month"
                trendUp={true}
              />
              <MetricCard
                title="Pending Orders"
                value={pendingOrdersCount}
                trend="-5%"
                subtext="processing faster"
                trendUp={true}
              />
              <MetricCard
                title="Low Stock SKUs"
                value={lowStockCount}
                trend="+2"
                subtext="needs attention"
                trendUp={false}
              />
              <MetricCard
                title="Gross Margin (Avg)"
                value="68%"
                trend="+1.2%"
                subtext="healthy"
                trendUp={true}
              />
            </div>

            <div className="grid grid-cols-1 lg:grid-cols-3 gap-6">
              <Card className="col-span-2 p-6">
                <div className="flex justify-between items-center mb-6">
                  <h3 className="font-bold text-slate-800">Recent Sales Velocity</h3>
                  <select className="text-sm border-slate-200 rounded-md text-slate-500">
                    <option>Last 7 Days</option>
                  </select>
                </div>
                {/* Simulated Chart */}
                <div className="h-64 flex items-end space-x-2">
                  {[45, 60, 35, 70, 85, 60, 75].map((h, i) => (
                    <div key={i} className="flex-1 bg-slate-100 rounded-t-lg relative group">
                      <div
                        className="absolute bottom-0 w-full bg-slate-800 rounded-t-lg transition-all duration-500 hover:bg-indigo-600"
                        style={{ height: `${h}%` }}
                      ></div>
                      <div className="opacity-0 group-hover:opacity-100 absolute -top-8 left-1/2 -translate-x-1/2 bg-slate-900 text-white text-xs py-1 px-2 rounded">
                        ${h * 100}
                      </div>
                    </div>
                  ))}
                </div>
                <div className="flex justify-between mt-2 text-xs text-slate-400">
                  <span>Mon</span><span>Tue</span><span>Wed</span><span>Thu</span><span>Fri</span><span>Sat</span><span>Sun</span>
                </div>
              </Card>

              <Card className="p-6">
                <h3 className="font-bold text-slate-800 mb-4">Stock Alerts</h3>
                <div className="space-y-4">
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).map(item => (
                    <div key={item.id} className="flex items-start p-3 bg-red-50 rounded-lg border border-red-100">
                      <AlertCircle className="w-5 h-5 text-red-500 mt-0.5 mr-3 flex-shrink-0" />
                      <div>
                        <p className="text-sm font-medium text-red-900">{item.name}</p>
                        <p className="text-xs text-red-700 mt-1">
                          Only {item.stock.warehouse + item.stock.store} units left (Safety: {item.safetyStock})
                        </p>
                        <button className="mt-2 text-xs font-medium text-red-600 hover:text-red-800">
                          Create PO &rarr;
                        </button>
                      </div>
                    </div>
                  ))}
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) >= i.safetyStock).slice(0, 2).map(item => (
                    <div key={item.id} className="flex items-center p-3 hover:bg-slate-50 rounded-lg transition-colors">
                      <div className="w-2 h-2 rounded-full bg-emerald-500 mr-3"></div>
                      <div className="flex-1">
                        <p className="text-sm font-medium text-slate-700">{item.name}</p>
                        <p className="text-xs text-slate-500">{item.stock.warehouse + item.stock.store} Units Available</p>
                      </div>
                    </div>
                  ))}
                </div>
              </Card>
            </div>
          </div>
        );

      case 'inventory':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex flex-col sm:flex-row justify-between items-start sm:items-center gap-4">
              <h2 className="text-xl font-bold text-slate-900">Inventory Management</h2>
              <div className="flex gap-2">
                <button
                  onClick={handleExport}
                  className="flex items-center px-4 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50"
                >
                  <Download className="w-4 h-4 mr-2" /> Export
                </button>
                <button
                  onClick={() => setIsAddModalOpen(true)}
                  className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800"
                >
                  <Plus className="w-4 h-4 mr-2" /> Add SKU
                </button>
              </div>
            </div>

            <Card className="overflow-hidden">
              <div className="p-4 border-b border-slate-100 bg-slate-50/50 flex flex-col sm:flex-row gap-4">
                <div className="relative flex-1">
                  <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
                  <input
                    type="text"
                    placeholder="Search SKUs, names, or categories..."
                    className="w-full pl-10 pr-4 py-2 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
                    value={searchTerm}
                    onChange={(e) => setSearchTerm(e.target.value)}
                  />
                </div>
                <button className="flex items-center px-3 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50">
                  <Filter className="w-4 h-4 mr-2" /> Filters
                </button>
              </div>

              <div className="overflow-x-auto">
                <table className="w-full text-left text-sm">
                  <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                    <tr>
                      <th className="px-6 py-4">Product / SKU</th>
                      <th className="px-6 py-4">Location Split</th>
                      <th className="px-6 py-4 text-right">Cost</th>
                      <th className="px-6 py-4 text-right">Price</th>
                      <th className="px-6 py-4 text-right">Margin</th>
                      <th className="px-6 py-4">Status</th>
                      <th className="px-6 py-4 text-right">Actions</th>
                    </tr>
                  </thead>
                  <tbody className="divide-y divide-slate-100">
                    {filteredInventory.map((item) => {
                      const total = item.stock.warehouse + item.stock.store;
                      const margin = ((item.price - item.cost) / item.price * 100).toFixed(0);
                      return (
                        <tr key={item.id} className="hover:bg-slate-50/50 transition-colors">
                          <td className="px-6 py-4">
                            <div className="font-medium text-slate-900">{item.name}</div>
                            <div className="text-xs text-slate-400 font-mono mt-0.5">{item.id}</div>
                          </td>
                          <td className="px-6 py-4">
                            <div className="flex flex-col gap-1 text-xs">
                              <span className="flex justify-between w-32"><span className="text-slate-500">Wholesale:</span> <span className="font-mono">{item.stock.warehouse}</span></span>
                              <span className="flex justify-between w-32"><span className="text-slate-500">Store:</span> <span className="font-mono">{item.stock.store}</span></span>
                              <div className="w-32 h-1 bg-slate-100 rounded-full mt-1 overflow-hidden">
                                <div className="h-full bg-slate-900" style={{ width: `${(total / (item.safetyStock * 3)) * 100}%` }}></div>
                              </div>
                            </div>
                          </td>
                          <td className="px-6 py-4 text-right font-mono text-slate-600">${item.cost.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right font-mono text-slate-900">${item.price.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right">
                            <span className={`inline-block px-2 py-0.5 rounded text-xs font-medium ${margin > 50 ? 'bg-emerald-100 text-emerald-700' : 'bg-amber-100 text-amber-700'}`}>
                              {margin}%
                            </span>
                          </td>
                          <td className="px-6 py-4">
                            <Badge status={total < item.safetyStock ? 'Low' : 'Normal'} />
                          </td>
                          <td className="px-6 py-4 text-right text-slate-400">
                            <button
                              onClick={() => setEditingItem(item)}
                              className="hover:text-slate-900"
                            >
                              Edit
                            </button>
                          </td>
                        </tr>
                      );
                    })}
                  </tbody>
                </table>
              </div>
            </Card>

            {/* Add SKU Modal */}
            <Modal
              isOpen={isAddModalOpen}
              onClose={() => setIsAddModalOpen(false)}
              title="Add New SKU"
            >
              <InventoryForm
                onSubmit={handleInventorySubmit}
                onCancel={() => setIsAddModalOpen(false)}
              />
            </Modal>

            {/* Edit SKU Modal */}
            <Modal
              isOpen={!!editingItem}
              onClose={() => setEditingItem(null)}
              title="Edit SKU"
            >
              {editingItem && (
                <InventoryForm
                  initialData={editingItem}
                  onSubmit={handleInventorySubmit}
                  onCancel={() => setEditingItem(null)}
                />
              )}
            </Modal>
          </div>
        );

      case 'orders':
        return (
          <div className="space-y-6 animate-fade-in">
             <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">All Orders</h2>
              <div className="text-sm text-slate-500">Syncing with Shopify & Amazon...</div>
            </div>
            <Card className="overflow-hidden">
              <table className="w-full text-left text-sm">
                <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                  <tr>
                    <th className="px-6 py-4">Order ID</th>
                    <th className="px-6 py-4">Customer</th>
                    <th className="px-6 py-4">Channel</th>
                    <th className="px-6 py-4">Date</th>
                    <th className="px-6 py-4 text-right">Total</th>
                    <th className="px-6 py-4">Status</th>
                    <th className="px-6 py-4 text-right">Action</th>
                  </tr>
                </thead>
                <tbody className="divide-y divide-slate-100">
                  {orders.map((order) => (
                    <tr key={order.id} className="hover:bg-slate-50/50">
                      <td className="px-6 py-4 font-mono font-medium text-slate-900">{order.id}</td>
                      <td className="px-6 py-4">{order.customer}</td>
                      <td className="px-6 py-4">
                        <span className="flex items-center gap-2">
                           <span className={`w-2 h-2 rounded-full ${order.channel === 'Shopify' ? 'bg-green-500' : order.channel === 'Amazon' ? 'bg-orange-500' : 'bg-blue-500'}`}></span>
                           {order.channel}
                        </span>
                      </td>
                      <td className="px-6 py-4 text-slate-500">{order.date}</td>
                      <td className="px-6 py-4 text-right font-medium">${order.total.toFixed(2)}</td>
                      <td className="px-6 py-4"><Badge status={order.status} /></td>
                      <td className="px-6 py-4 text-right">
                        {order.status === 'Pending' && (
                          <button
                            onClick={() => fulfillOrder(order.id)}
                            className="text-xs bg-slate-900 text-white px-3 py-1.5 rounded-md hover:bg-slate-700 transition-colors"
                          >
                            Ship
                          </button>
                        )}
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </Card>
          </div>
        );

      case 'purchasing':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">Purchase Orders</h2>
              <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                <Plus className="w-4 h-4 mr-2" /> New PO
              </button>
            </div>
             <div className="grid gap-6">
                {purchaseOrders.map((po) => (
                  <Card key={po.id} className="p-0 overflow-hidden flex flex-col md:flex-row">
                     <div className="p-6 flex-1">
                        <div className="flex justify-between items-start mb-4">
                          <div>
                            <div className="flex items-center gap-3">
                              <h3 className="font-bold text-lg text-slate-900">{po.id}</h3>
                              <Badge status={po.status} />
                            </div>
                            <p className="text-slate-500 text-sm mt-1">Vendor: <span className="font-medium text-slate-700">{po.vendor}</span></p>
                          </div>
                          <div className="text-right">
                             <div className="font-mono font-bold text-lg">${po.total.toFixed(2)}</div>
                             <div className="text-xs text-slate-400">Total Value</div>
                          </div>
                        </div>
                        <div className="flex items-center gap-6 text-sm text-slate-600">
                           <div className="flex items-center gap-2">
                              <Truck className="w-4 h-4" />
                              <span>Expected: {po.expected}</span>
                           </div>
                           <div className="flex items-center gap-2">
                              <DollarSign className="w-4 h-4" />
                              <span>Payment: Net 30</span>
                           </div>
                        </div>
                     </div>
                     {po.status !== 'Received' && (
                       <div className="bg-slate-50 p-6 flex items-center border-t md:border-t-0 md:border-l border-slate-100">
                          <button
                            onClick={() => receivePO(po.id)}
                            className="w-full md:w-auto flex items-center justify-center px-6 py-2 bg-emerald-600 text-white rounded-lg hover:bg-emerald-700 transition-colors font-medium text-sm"
                          >
                            <CheckCircle className="w-4 h-4 mr-2" />
                            Receive Goods
                          </button>
                       </div>
                     )}
                  </Card>
                ))}
             </div>
          </div>
        );

      default:
        return <div className="p-10 text-center text-slate-400">Module under construction</div>;
    }
  };

  return (
    <div className="flex h-screen bg-slate-50 font-sans text-slate-900 overflow-hidden">
      {/* Added style block to define the animation which was missing in Tailwind default config */}
      <style>{`
        @keyframes fadeIn {
          from { opacity: 0; transform: translateY(10px); }
          to { opacity: 1; transform: translateY(0); }
        }
        .animate-fade-in {
          animation: fadeIn 0.4s ease-out forwards;
        }
      `}</style>

      {/* Sidebar for Desktop */}
      <aside className="hidden md:flex flex-col w-64 bg-white border-r border-slate-200 h-full">
        <div className="p-6 flex items-center border-b border-slate-100">
          <div className="w-8 h-8 bg-indigo-600 rounded-lg flex items-center justify-center mr-3 shadow-sm">
            <Package className="w-5 h-5 text-white" />
          </div>
          <span className="font-bold text-lg tracking-tight">ZeroERP</span>
        </div>
        <div className="flex-1 overflow-y-auto py-6 px-4">
          <Navigation />
        </div>
        <div className="p-4 border-t border-slate-100">
          <div className="flex items-center p-2 rounded-lg bg-slate-50 border border-slate-100">
            <div className="w-8 h-8 rounded-full bg-slate-200 flex items-center justify-center text-xs font-bold text-slate-600">
              JS
            </div>
            <div className="ml-3">
              <p className="text-xs font-bold text-slate-900">John Smith</p>
              <p className="text-[10px] text-slate-500">Admin</p>
            </div>
          </div>
        </div>
      </aside>

      {/* Mobile Sidebar Overlay */}
      {isMobileMenuOpen && (
        <div className="fixed inset-0 z-50 bg-slate-900/50 md:hidden" onClick={() => setIsMobileMenuOpen(false)}>
          <div className="absolute left-0 top-0 bottom-0 w-64 bg-white p-4" onClick={e => e.stopPropagation()}>
             <div className="flex justify-between items-center mb-6">
                <span className="font-bold text-lg">ZeroERP</span>
                <button onClick={() => setIsMobileMenuOpen(false)}><X className="w-6 h-6" /></button>
             </div>
             <Navigation />
          </div>
        </div>
      )}

      {/* Main Content Area */}
      <main className="flex-1 flex flex-col h-full overflow-hidden">
        {/* Top Header */}
        <header className="h-16 bg-white border-b border-slate-200 flex justify-between items-center px-6">
          <div className="flex items-center md:hidden">
            <button onClick={() => setIsMobileMenuOpen(true)} className="mr-4">
              <Menu className="w-6 h-6 text-slate-600" />
            </button>
          </div>

          <div className="hidden md:flex flex-1 max-w-md relative">
            <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
            <input
              type="text"
              placeholder="Command + K to search..."
              className="w-full pl-10 pr-4 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-indigo-500/20 focus:border-indigo-500 transition-all"
            />
            <div className="absolute right-3 top-1/2 -translate-y-1/2 flex gap-1">
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">Cmd</span>
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">K</span>
            </div>
          </div>

          <div className="flex items-center gap-4">
            <button className="relative p-2 text-slate-400 hover:bg-slate-50 rounded-lg transition-colors">
              <Bell className="w-5 h-5" />
              <span className="absolute top-2 right-2 w-2 h-2 bg-rose-500 rounded-full border-2 border-white"></span>
            </button>
          </div>
        </header>

        {/* Scrollable Page Content */}
        <div className="flex-1 overflow-auto p-4 md:p-8">
           <div className="max-w-7xl mx-auto">
             {renderContent()}
           </div>
        </div>
      </main>
    </div>
  );
}


Overwriting ZeroERP.js


**Reasoning**:
I will update `ZeroERP.js` to include the `Modal` and `InventoryForm` components and integrate them into the `App` component. This involves adding state for managing modal visibility (`isAddModalOpen`, `editingItem`), creating the form logic, and updating the UI to render these modals in the 'inventory' view, while preserving the existing export functionality.



In [3]:
%%writefile ZeroERP.js
import React, { useState, useMemo } from 'react';
import {
  LayoutDashboard,
  Package,
  ShoppingCart,
  Truck,
  Settings,
  Search,
  Bell,
  Menu,
  X,
  ArrowUpRight,
  ArrowDownRight,
  Plus,
  Filter,
  Download,
  CheckCircle,
  AlertCircle,
  DollarSign,
  BarChart3
} from 'lucide-react';

// --- Mock Data Generators ---

const INITIAL_INVENTORY = [
  { id: 'SKU-001', name: 'Ceramic Pour-Over Set', category: 'Coffee', stock: { warehouse: 45, store: 12 }, safetyStock: 20, cost: 15.00, price: 45.00, vendor: 'CeramicsCo' },
  { id: 'SKU-002', name: 'Ethiopian Yirgacheffe (12oz)', category: 'Coffee', stock: { warehouse: 120, store: 50 }, safetyStock: 50, cost: 8.50, price: 22.00, vendor: 'GlobalBeans' },
  { id: 'SKU-003', name: 'Gooseneck Kettle (Matte Black)', category: 'Equipment', stock: { warehouse: 5, store: 2 }, safetyStock: 10, cost: 35.00, price: 85.00, vendor: 'HeatSync' },
  { id: 'SKU-004', name: 'Paper Filters (100ct)', category: 'Consumables', stock: { warehouse: 500, store: 100 }, safetyStock: 200, cost: 2.00, price: 8.00, vendor: 'PaperWorks' },
  { id: 'SKU-005', name: 'Barista Apron (Canvas)', category: 'Apparel', stock: { warehouse: 25, store: 5 }, safetyStock: 15, cost: 12.00, price: 40.00, vendor: 'TextileBros' },
  { id: 'SKU-006', name: 'Cold Brew Kit', category: 'Bundles', stock: { warehouse: 15, store: 0 }, safetyStock: 10, cost: 18.00, price: 55.00, vendor: 'Internal' },
];

const INITIAL_ORDERS = [
  { id: 'ORD-7782', customer: 'Alice Freeman', channel: 'Shopify', total: 135.00, status: 'Pending', date: '2023-10-24', items: 3 },
  { id: 'ORD-7781', customer: 'Mark Stone', channel: 'Amazon', total: 22.00, status: 'Shipped', date: '2023-10-24', items: 1 },
  { id: 'ORD-7780', customer: 'Cafe Lux (Wholesale)', channel: 'B2B Portal', total: 450.00, status: 'Shipped', date: '2023-10-23', items: 20 },
  { id: 'ORD-7779', customer: 'Sarah Connor', channel: 'Shopify', total: 85.00, status: 'Delivered', date: '2023-10-23', items: 1 },
];

const INITIAL_PO = [
  { id: 'PO-9001', vendor: 'HeatSync', status: 'Ordered', expected: '2023-11-01', total: 1500.00 },
  { id: 'PO-9002', vendor: 'GlobalBeans', status: 'Received', expected: '2023-10-20', total: 850.00 },
];

// --- Components ---

const Card = ({ children, className = "" }) => (
  <div className={`bg-white rounded-xl border border-slate-200 shadow-sm ${className}`}>
    {children}
  </div>
);

const Badge = ({ status }) => {
  const styles = {
    Pending: "bg-amber-100 text-amber-700",
    Shipped: "bg-blue-100 text-blue-700",
    Delivered: "bg-emerald-100 text-emerald-700",
    Ordered: "bg-purple-100 text-purple-700",
    Received: "bg-slate-100 text-slate-700",
    Low: "bg-red-100 text-red-700",
    Normal: "bg-emerald-100 text-emerald-700",
  };
  return (
    <span className={`px-2.5 py-0.5 rounded-full text-xs font-medium ${styles[status] || "bg-gray-100 text-gray-800"}`}>
      {status}
    </span>
  );
};

const MetricCard = ({ title, value, subtext, trend, trendUp }) => (
  <Card className="p-5">
    <div className="flex justify-between items-start">
      <div>
        <p className="text-sm font-medium text-slate-500">{title}</p>
        <h3 className="text-2xl font-bold text-slate-900 mt-1">{value}</h3>
      </div>
      <div className={`p-2 rounded-lg ${trendUp ? 'bg-emerald-50' : 'bg-rose-50'}`}>
        {trendUp ? <ArrowUpRight className="w-5 h-5 text-emerald-600" /> : <ArrowDownRight className="w-5 h-5 text-rose-600" />}
      </div>
    </div>
    <div className="mt-4 flex items-center text-xs">
      <span className={`font-medium ${trendUp ? 'text-emerald-600' : 'text-rose-600'}`}>
        {trend}
      </span>
      <span className="text-slate-400 ml-1">{subtext}</span>
    </div>
  </Card>
);

// --- Modal Component ---
const Modal = ({ isOpen, onClose, title, children }) => {
  if (!isOpen) return null;
  return (
    <div className="fixed inset-0 z-50 flex items-center justify-center bg-slate-900/50 backdrop-blur-sm p-4 animate-fade-in">
      <div className="bg-white rounded-xl shadow-xl w-full max-w-lg max-h-[90vh] overflow-y-auto">
        <div className="flex justify-between items-center p-4 border-b border-slate-100 sticky top-0 bg-white z-10">
          <h3 className="font-bold text-lg text-slate-800">{title}</h3>
          <button onClick={onClose} className="text-slate-400 hover:text-slate-600">
            <X className="w-5 h-5" />
          </button>
        </div>
        <div className="p-6">
          {children}
        </div>
      </div>
    </div>
  );
};

// --- Inventory Form Component ---
const InventoryForm = ({ initialData = {}, onSubmit, onCancel }) => {
  const [formData, setFormData] = useState({
    name: initialData.name || '',
    category: initialData.category || '',
    warehouse: initialData.stock?.warehouse || 0,
    store: initialData.stock?.store || 0,
    safetyStock: initialData.safetyStock || 0,
    cost: initialData.cost || 0,
    price: initialData.price || 0,
    vendor: initialData.vendor || ''
  });

  const handleChange = (e) => {
    const { name, value } = e.target;
    setFormData(prev => ({
      ...prev,
      [name]: ['warehouse', 'store', 'safetyStock', 'cost', 'price'].includes(name) ? parseFloat(value) || 0 : value
    }));
  };

  const handleSubmit = (e) => {
    e.preventDefault();
    onSubmit(formData);
  };

  const InputField = ({ label, name, type = "text", step }) => (
    <div className="space-y-1">
      <label className="text-xs font-medium text-slate-500 uppercase tracking-wider">{label}</label>
      <input
        type={type}
        name={name}
        step={step}
        value={formData[name]}
        onChange={handleChange}
        className="w-full px-3 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
        required
      />
    </div>
  );

  return (
    <form onSubmit={handleSubmit} className="space-y-4">
      <InputField label="Product Name" name="name" />
      <div className="grid grid-cols-2 gap-4">
        <InputField label="Category" name="category" />
        <InputField label="Vendor" name="vendor" />
      </div>
      <div className="grid grid-cols-2 gap-4">
        <InputField label="Cost ($)" name="cost" type="number" step="0.01" />
        <InputField label="Price ($)" name="price" type="number" step="0.01" />
      </div>
      <div className="p-4 bg-slate-50 rounded-lg space-y-4 border border-slate-100">
        <h4 className="text-sm font-bold text-slate-700">Stock Levels</h4>
        <div className="grid grid-cols-3 gap-3">
          <InputField label="Warehouse" name="warehouse" type="number" />
          <InputField label="Store" name="store" type="number" />
          <InputField label="Safety Stock" name="safetyStock" type="number" />
        </div>
      </div>

      <div className="flex justify-end gap-3 pt-4 border-t border-slate-100">
        <button type="button" onClick={onCancel} className="px-4 py-2 text-sm font-medium text-slate-600 bg-white border border-slate-200 rounded-lg hover:bg-slate-50">Cancel</button>
        <button type="submit" className="px-4 py-2 text-sm font-medium text-white bg-slate-900 rounded-lg hover:bg-slate-800">Save Changes</button>
      </div>
    </form>
  );
};

// --- Main Application ---

export default function App() {
  const [activeTab, setActiveTab] = useState('dashboard');
  const [isMobileMenuOpen, setIsMobileMenuOpen] = useState(false);

  // App State
  const [inventory, setInventory] = useState(INITIAL_INVENTORY);
  const [orders, setOrders] = useState(INITIAL_ORDERS);
  const [purchaseOrders, setPurchaseOrders] = useState(INITIAL_PO);
  const [searchTerm, setSearchTerm] = useState('');

  // Modal State
  const [isAddModalOpen, setIsAddModalOpen] = useState(false);
  const [editingItem, setEditingItem] = useState(null);

  // Derived Metrics
  const totalStockValue = useMemo(() =>
    inventory.reduce((acc, item) => acc + ((item.stock.warehouse + item.stock.store) * item.cost), 0),
  [inventory]);

  const lowStockCount = useMemo(() =>
    inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).length,
  [inventory]);

  const pendingOrdersCount = useMemo(() =>
    orders.filter(o => o.status === 'Pending').length,
  [orders]);

  // Actions
  const fulfillOrder = (orderId) => {
    setOrders(orders.map(o => o.id === orderId ? { ...o, status: 'Shipped' } : o));
    alert(`Order ${orderId} marked as Shipped. (Simulation: Inventory would be adjusted here)`);
  };

  const receivePO = (poId) => {
    setPurchaseOrders(purchaseOrders.map(p => p.id === poId ? { ...p, status: 'Received' } : p));
    alert(`PO ${poId} received. (Simulation: Stock levels would be updated here)`);
  };

  const handleExport = () => {
    const headers = ['ID,Name,Category,Warehouse Stock,Store Stock,Safety Stock,Cost,Price,Vendor'];
    const rows = inventory.map(item => {
      return [
        item.id,
        `"${item.name}"`,
        item.category,
        item.stock.warehouse,
        item.stock.store,
        item.safetyStock,
        item.cost.toFixed(2),
        item.price.toFixed(2),
        item.vendor
      ].join(',');
    });
    const csvString = [headers, ...rows].join('\n');
    const blob = new Blob([csvString], { type: 'text/csv' });
    const url = URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = 'inventory_export.csv';
    document.body.appendChild(a);
    a.click();
    document.body.removeChild(a);
    URL.revokeObjectURL(url);
  };

  // Placeholder for modal submit
  const handleInventorySubmit = (data) => {
    // This will be implemented fully in the next step
    console.log("Form Submitted:", data);
    alert("Item saved! (State update logic coming soon)");
    setIsAddModalOpen(false);
    setEditingItem(null);
  };

  const filteredInventory = inventory.filter(item =>
    item.name.toLowerCase().includes(searchTerm.toLowerCase()) ||
    item.id.toLowerCase().includes(searchTerm.toLowerCase())
  );

  const Navigation = () => (
    <nav className="space-y-1">
      {[
        { id: 'dashboard', label: 'Dashboard', icon: LayoutDashboard },
        { id: 'inventory', label: 'Inventory', icon: Package },
        { id: 'orders', label: 'Orders', icon: ShoppingCart },
        { id: 'purchasing', label: 'Purchasing', icon: Truck },
        { id: 'reports', label: 'Reports', icon: BarChart3 },
        { id: 'settings', label: 'Settings', icon: Settings },
      ].map((item) => (
        <button
          key={item.id}
          onClick={() => {
            setActiveTab(item.id);
            setIsMobileMenuOpen(false);
          }}
          className={`w-full flex items-center px-4 py-3 text-sm font-medium rounded-lg transition-colors ${
            activeTab === item.id
              ? 'bg-slate-900 text-white'
              : 'text-slate-600 hover:bg-slate-100'
          }`}
        >
          <item.icon className="w-5 h-5 mr-3" />
          {item.label}
          {item.id === 'orders' && pendingOrdersCount > 0 && (
            <span className="ml-auto bg-rose-500 text-white text-xs font-bold px-2 py-0.5 rounded-full">
              {pendingOrdersCount}
            </span>
          )}
        </button>
      ))}
    </nav>
  );

  const renderContent = () => {
    switch(activeTab) {
      case 'dashboard':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4">
              <MetricCard
                title="Total Stock Value"
                value={`$${totalStockValue.toLocaleString()}`}
                trend="+12%"
                subtext="vs last month"
                trendUp={true}
              />
              <MetricCard
                title="Pending Orders"
                value={pendingOrdersCount}
                trend="-5%"
                subtext="processing faster"
                trendUp={true}
              />
              <MetricCard
                title="Low Stock SKUs"
                value={lowStockCount}
                trend="+2"
                subtext="needs attention"
                trendUp={false}
              />
              <MetricCard
                title="Gross Margin (Avg)"
                value="68%"
                trend="+1.2%"
                subtext="healthy"
                trendUp={true}
              />
            </div>

            <div className="grid grid-cols-1 lg:grid-cols-3 gap-6">
              <Card className="col-span-2 p-6">
                <div className="flex justify-between items-center mb-6">
                  <h3 className="font-bold text-slate-800">Recent Sales Velocity</h3>
                  <select className="text-sm border-slate-200 rounded-md text-slate-500">
                    <option>Last 7 Days</option>
                  </select>
                </div>
                {/* Simulated Chart */}
                <div className="h-64 flex items-end space-x-2">
                  {[45, 60, 35, 70, 85, 60, 75].map((h, i) => (
                    <div key={i} className="flex-1 bg-slate-100 rounded-t-lg relative group">
                      <div
                        className="absolute bottom-0 w-full bg-slate-800 rounded-t-lg transition-all duration-500 hover:bg-indigo-600"
                        style={{ height: `${h}%` }}
                      ></div>
                      <div className="opacity-0 group-hover:opacity-100 absolute -top-8 left-1/2 -translate-x-1/2 bg-slate-900 text-white text-xs py-1 px-2 rounded">
                        ${h * 100}
                      </div>
                    </div>
                  ))}
                </div>
                <div className="flex justify-between mt-2 text-xs text-slate-400">
                  <span>Mon</span><span>Tue</span><span>Wed</span><span>Thu</span><span>Fri</span><span>Sat</span><span>Sun</span>
                </div>
              </Card>

              <Card className="p-6">
                <h3 className="font-bold text-slate-800 mb-4">Stock Alerts</h3>
                <div className="space-y-4">
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).map(item => (
                    <div key={item.id} className="flex items-start p-3 bg-red-50 rounded-lg border border-red-100">
                      <AlertCircle className="w-5 h-5 text-red-500 mt-0.5 mr-3 flex-shrink-0" />
                      <div>
                        <p className="text-sm font-medium text-red-900">{item.name}</p>
                        <p className="text-xs text-red-700 mt-1">
                          Only {item.stock.warehouse + item.stock.store} units left (Safety: {item.safetyStock})
                        </p>
                        <button className="mt-2 text-xs font-medium text-red-600 hover:text-red-800">
                          Create PO &rarr;
                        </button>
                      </div>
                    </div>
                  ))}
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) >= i.safetyStock).slice(0, 2).map(item => (
                    <div key={item.id} className="flex items-center p-3 hover:bg-slate-50 rounded-lg transition-colors">
                      <div className="w-2 h-2 rounded-full bg-emerald-500 mr-3"></div>
                      <div className="flex-1">
                        <p className="text-sm font-medium text-slate-700">{item.name}</p>
                        <p className="text-xs text-slate-500">{item.stock.warehouse + item.stock.store} Units Available</p>
                      </div>
                    </div>
                  ))}
                </div>
              </Card>
            </div>
          </div>
        );

      case 'inventory':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex flex-col sm:flex-row justify-between items-start sm:items-center gap-4">
              <h2 className="text-xl font-bold text-slate-900">Inventory Management</h2>
              <div className="flex gap-2">
                <button
                  onClick={handleExport}
                  className="flex items-center px-4 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50"
                >
                  <Download className="w-4 h-4 mr-2" /> Export
                </button>
                <button
                  onClick={() => setIsAddModalOpen(true)}
                  className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800"
                >
                  <Plus className="w-4 h-4 mr-2" /> Add SKU
                </button>
              </div>
            </div>

            <Card className="overflow-hidden">
              <div className="p-4 border-b border-slate-100 bg-slate-50/50 flex flex-col sm:flex-row gap-4">
                <div className="relative flex-1">
                  <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
                  <input
                    type="text"
                    placeholder="Search SKUs, names, or categories..."
                    className="w-full pl-10 pr-4 py-2 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
                    value={searchTerm}
                    onChange={(e) => setSearchTerm(e.target.value)}
                  />
                </div>
                <button className="flex items-center px-3 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50">
                  <Filter className="w-4 h-4 mr-2" /> Filters
                </button>
              </div>

              <div className="overflow-x-auto">
                <table className="w-full text-left text-sm">
                  <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                    <tr>
                      <th className="px-6 py-4">Product / SKU</th>
                      <th className="px-6 py-4">Location Split</th>
                      <th className="px-6 py-4 text-right">Cost</th>
                      <th className="px-6 py-4 text-right">Price</th>
                      <th className="px-6 py-4 text-right">Margin</th>
                      <th className="px-6 py-4">Status</th>
                      <th className="px-6 py-4 text-right">Actions</th>
                    </tr>
                  </thead>
                  <tbody className="divide-y divide-slate-100">
                    {filteredInventory.map((item) => {
                      const total = item.stock.warehouse + item.stock.store;
                      const margin = ((item.price - item.cost) / item.price * 100).toFixed(0);
                      return (
                        <tr key={item.id} className="hover:bg-slate-50/50 transition-colors">
                          <td className="px-6 py-4">
                            <div className="font-medium text-slate-900">{item.name}</div>
                            <div className="text-xs text-slate-400 font-mono mt-0.5">{item.id}</div>
                          </td>
                          <td className="px-6 py-4">
                            <div className="flex flex-col gap-1 text-xs">
                              <span className="flex justify-between w-32"><span className="text-slate-500">Wholesale:</span> <span className="font-mono">{item.stock.warehouse}</span></span>
                              <span className="flex justify-between w-32"><span className="text-slate-500">Store:</span> <span className="font-mono">{item.stock.store}</span></span>
                              <div className="w-32 h-1 bg-slate-100 rounded-full mt-1 overflow-hidden">
                                <div className="h-full bg-slate-900" style={{ width: `${(total / (item.safetyStock * 3)) * 100}%` }}></div>
                              </div>
                            </div>
                          </td>
                          <td className="px-6 py-4 text-right font-mono text-slate-600">${item.cost.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right font-mono text-slate-900">${item.price.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right">
                            <span className={`inline-block px-2 py-0.5 rounded text-xs font-medium ${margin > 50 ? 'bg-emerald-100 text-emerald-700' : 'bg-amber-100 text-amber-700'}`}>
                              {margin}%
                            </span>
                          </td>
                          <td className="px-6 py-4">
                            <Badge status={total < item.safetyStock ? 'Low' : 'Normal'} />
                          </td>
                          <td className="px-6 py-4 text-right text-slate-400">
                            <button
                              onClick={() => setEditingItem(item)}
                              className="hover:text-slate-900"
                            >
                              Edit
                            </button>
                          </td>
                        </tr>
                      );
                    })}
                  </tbody>
                </table>
              </div>
            </Card>

            {/* Add SKU Modal */}
            <Modal
              isOpen={isAddModalOpen}
              onClose={() => setIsAddModalOpen(false)}
              title="Add New SKU"
            >
              <InventoryForm
                onSubmit={handleInventorySubmit}
                onCancel={() => setIsAddModalOpen(false)}
              />
            </Modal>

            {/* Edit SKU Modal */}
            <Modal
              isOpen={!!editingItem}
              onClose={() => setEditingItem(null)}
              title="Edit SKU"
            >
              {editingItem && (
                <InventoryForm
                  initialData={editingItem}
                  onSubmit={handleInventorySubmit}
                  onCancel={() => setEditingItem(null)}
                />
              )}
            </Modal>
          </div>
        );

      case 'orders':
        return (
          <div className="space-y-6 animate-fade-in">
             <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">All Orders</h2>
              <div className="text-sm text-slate-500">Syncing with Shopify & Amazon...</div>
            </div>
            <Card className="overflow-hidden">
              <table className="w-full text-left text-sm">
                <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                  <tr>
                    <th className="px-6 py-4">Order ID</th>
                    <th className="px-6 py-4">Customer</th>
                    <th className="px-6 py-4">Channel</th>
                    <th className="px-6 py-4">Date</th>
                    <th className="px-6 py-4 text-right">Total</th>
                    <th className="px-6 py-4">Status</th>
                    <th className="px-6 py-4 text-right">Action</th>
                  </tr>
                </thead>
                <tbody className="divide-y divide-slate-100">
                  {orders.map((order) => (
                    <tr key={order.id} className="hover:bg-slate-50/50">
                      <td className="px-6 py-4 font-mono font-medium text-slate-900">{order.id}</td>
                      <td className="px-6 py-4">{order.customer}</td>
                      <td className="px-6 py-4">
                        <span className="flex items-center gap-2">
                           <span className={`w-2 h-2 rounded-full ${order.channel === 'Shopify' ? 'bg-green-500' : order.channel === 'Amazon' ? 'bg-orange-500' : 'bg-blue-500'}`}></span>
                           {order.channel}
                        </span>
                      </td>
                      <td className="px-6 py-4 text-slate-500">{order.date}</td>
                      <td className="px-6 py-4 text-right font-medium">${order.total.toFixed(2)}</td>
                      <td className="px-6 py-4"><Badge status={order.status} /></td>
                      <td className="px-6 py-4 text-right">
                        {order.status === 'Pending' && (
                          <button
                            onClick={() => fulfillOrder(order.id)}
                            className="text-xs bg-slate-900 text-white px-3 py-1.5 rounded-md hover:bg-slate-700 transition-colors"
                          >
                            Ship
                          </button>
                        )}
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </Card>
          </div>
        );

      case 'purchasing':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">Purchase Orders</h2>
              <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                <Plus className="w-4 h-4 mr-2" /> New PO
              </button>
            </div>
             <div className="grid gap-6">
                {purchaseOrders.map((po) => (
                  <Card key={po.id} className="p-0 overflow-hidden flex flex-col md:flex-row">
                     <div className="p-6 flex-1">
                        <div className="flex justify-between items-start mb-4">
                          <div>
                            <div className="flex items-center gap-3">
                              <h3 className="font-bold text-lg text-slate-900">{po.id}</h3>
                              <Badge status={po.status} />
                            </div>
                            <p className="text-slate-500 text-sm mt-1">Vendor: <span className="font-medium text-slate-700">{po.vendor}</span></p>
                          </div>
                          <div className="text-right">
                             <div className="font-mono font-bold text-lg">${po.total.toFixed(2)}</div>
                             <div className="text-xs text-slate-400">Total Value</div>
                          </div>
                        </div>
                        <div className="flex items-center gap-6 text-sm text-slate-600">
                           <div className="flex items-center gap-2">
                              <Truck className="w-4 h-4" />
                              <span>Expected: {po.expected}</span>
                           </div>
                           <div className="flex items-center gap-2">
                              <DollarSign className="w-4 h-4" />
                              <span>Payment: Net 30</span>
                           </div>
                        </div>
                     </div>
                     {po.status !== 'Received' && (
                       <div className="bg-slate-50 p-6 flex items-center border-t md:border-t-0 md:border-l border-slate-100">
                          <button
                            onClick={() => receivePO(po.id)}
                            className="w-full md:w-auto flex items-center justify-center px-6 py-2 bg-emerald-600 text-white rounded-lg hover:bg-emerald-700 transition-colors font-medium text-sm"
                          >
                            <CheckCircle className="w-4 h-4 mr-2" />
                            Receive Goods
                          </button>
                       </div>
                     )}
                  </Card>
                ))}
             </div>
          </div>
        );

      default:
        return <div className="p-10 text-center text-slate-400">Module under construction</div>;
    }
  };

  return (
    <div className="flex h-screen bg-slate-50 font-sans text-slate-900 overflow-hidden">
      {/* Added style block to define the animation which was missing in Tailwind default config */}
      <style>{`
        @keyframes fadeIn {
          from { opacity: 0; transform: translateY(10px); }
          to { opacity: 1; transform: translateY(0); }
        }
        .animate-fade-in {
          animation: fadeIn 0.4s ease-out forwards;
        }
      `}</style>

      {/* Sidebar for Desktop */}
      <aside className="hidden md:flex flex-col w-64 bg-white border-r border-slate-200 h-full">
        <div className="p-6 flex items-center border-b border-slate-100">
          <div className="w-8 h-8 bg-indigo-600 rounded-lg flex items-center justify-center mr-3 shadow-sm">
            <Package className="w-5 h-5 text-white" />
          </div>
          <span className="font-bold text-lg tracking-tight">ZeroERP</span>
        </div>
        <div className="flex-1 overflow-y-auto py-6 px-4">
          <Navigation />
        </div>
        <div className="p-4 border-t border-slate-100">
          <div className="flex items-center p-2 rounded-lg bg-slate-50 border border-slate-100">
            <div className="w-8 h-8 rounded-full bg-slate-200 flex items-center justify-center text-xs font-bold text-slate-600">
              JS
            </div>
            <div className="ml-3">
              <p className="text-xs font-bold text-slate-900">John Smith</p>
              <p className="text-[10px] text-slate-500">Admin</p>
            </div>
          </div>
        </div>
      </aside>

      {/* Mobile Sidebar Overlay */}
      {isMobileMenuOpen && (
        <div className="fixed inset-0 z-50 bg-slate-900/50 md:hidden" onClick={() => setIsMobileMenuOpen(false)}>
          <div className="absolute left-0 top-0 bottom-0 w-64 bg-white p-4" onClick={e => e.stopPropagation()}>
             <div className="flex justify-between items-center mb-6">
                <span className="font-bold text-lg">ZeroERP</span>
                <button onClick={() => setIsMobileMenuOpen(false)}><X className="w-6 h-6" /></button>
             </div>
             <Navigation />
          </div>
        </div>
      )}

      {/* Main Content Area */}
      <main className="flex-1 flex flex-col h-full overflow-hidden">
        {/* Top Header */}
        <header className="h-16 bg-white border-b border-slate-200 flex justify-between items-center px-6">
          <div className="flex items-center md:hidden">
            <button onClick={() => setIsMobileMenuOpen(true)} className="mr-4">
              <Menu className="w-6 h-6 text-slate-600" />
            </button>
          </div>

          <div className="hidden md:flex flex-1 max-w-md relative">
            <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
            <input
              type="text"
              placeholder="Command + K to search..."
              className="w-full pl-10 pr-4 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-indigo-500/20 focus:border-indigo-500 transition-all"
            />
            <div className="absolute right-3 top-1/2 -translate-y-1/2 flex gap-1">
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">Cmd</span>
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">K</span>
            </div>
          </div>

          <div className="flex items-center gap-4">
            <button className="relative p-2 text-slate-400 hover:bg-slate-50 rounded-lg transition-colors">
              <Bell className="w-5 h-5" />
              <span className="absolute top-2 right-2 w-2 h-2 bg-rose-500 rounded-full border-2 border-white"></span>
            </button>
          </div>
        </header>

        {/* Scrollable Page Content */}
        <div className="flex-1 overflow-auto p-4 md:p-8">
           <div className="max-w-7xl mx-auto">
             {renderContent()}
           </div>
        </div>
      </main>
    </div>
  );
}


Overwriting ZeroERP.js


## Integrate Inventory Actions

### Subtask:
Wire up the new modals to the main application state, allowing users to create new products and update existing ones.


**Reasoning**:
I will update `ZeroERP.js` to implement the logic for `handleInventorySubmit` within the `App` component. This involves distinguishing between adding and editing items, formatting the data to match the inventory schema (specifically nesting stock levels), updating the `inventory` state, and managing the modal open/close states.



In [4]:
%%writefile ZeroERP.js
import React, { useState, useMemo } from 'react';
import {
  LayoutDashboard,
  Package,
  ShoppingCart,
  Truck,
  Settings,
  Search,
  Bell,
  Menu,
  X,
  ArrowUpRight,
  ArrowDownRight,
  Plus,
  Filter,
  Download,
  CheckCircle,
  AlertCircle,
  DollarSign,
  BarChart3
} from 'lucide-react';

// --- Mock Data Generators ---

const INITIAL_INVENTORY = [
  { id: 'SKU-001', name: 'Ceramic Pour-Over Set', category: 'Coffee', stock: { warehouse: 45, store: 12 }, safetyStock: 20, cost: 15.00, price: 45.00, vendor: 'CeramicsCo' },
  { id: 'SKU-002', name: 'Ethiopian Yirgacheffe (12oz)', category: 'Coffee', stock: { warehouse: 120, store: 50 }, safetyStock: 50, cost: 8.50, price: 22.00, vendor: 'GlobalBeans' },
  { id: 'SKU-003', name: 'Gooseneck Kettle (Matte Black)', category: 'Equipment', stock: { warehouse: 5, store: 2 }, safetyStock: 10, cost: 35.00, price: 85.00, vendor: 'HeatSync' },
  { id: 'SKU-004', name: 'Paper Filters (100ct)', category: 'Consumables', stock: { warehouse: 500, store: 100 }, safetyStock: 200, cost: 2.00, price: 8.00, vendor: 'PaperWorks' },
  { id: 'SKU-005', name: 'Barista Apron (Canvas)', category: 'Apparel', stock: { warehouse: 25, store: 5 }, safetyStock: 15, cost: 12.00, price: 40.00, vendor: 'TextileBros' },
  { id: 'SKU-006', name: 'Cold Brew Kit', category: 'Bundles', stock: { warehouse: 15, store: 0 }, safetyStock: 10, cost: 18.00, price: 55.00, vendor: 'Internal' },
];

const INITIAL_ORDERS = [
  { id: 'ORD-7782', customer: 'Alice Freeman', channel: 'Shopify', total: 135.00, status: 'Pending', date: '2023-10-24', items: 3 },
  { id: 'ORD-7781', customer: 'Mark Stone', channel: 'Amazon', total: 22.00, status: 'Shipped', date: '2023-10-24', items: 1 },
  { id: 'ORD-7780', customer: 'Cafe Lux (Wholesale)', channel: 'B2B Portal', total: 450.00, status: 'Shipped', date: '2023-10-23', items: 20 },
  { id: 'ORD-7779', customer: 'Sarah Connor', channel: 'Shopify', total: 85.00, status: 'Delivered', date: '2023-10-23', items: 1 },
];

const INITIAL_PO = [
  { id: 'PO-9001', vendor: 'HeatSync', status: 'Ordered', expected: '2023-11-01', total: 1500.00 },
  { id: 'PO-9002', vendor: 'GlobalBeans', status: 'Received', expected: '2023-10-20', total: 850.00 },
];

// --- Components ---

const Card = ({ children, className = "" }) => (
  <div className={`bg-white rounded-xl border border-slate-200 shadow-sm ${className}`}>
    {children}
  </div>
);

const Badge = ({ status }) => {
  const styles = {
    Pending: "bg-amber-100 text-amber-700",
    Shipped: "bg-blue-100 text-blue-700",
    Delivered: "bg-emerald-100 text-emerald-700",
    Ordered: "bg-purple-100 text-purple-700",
    Received: "bg-slate-100 text-slate-700",
    Low: "bg-red-100 text-red-700",
    Normal: "bg-emerald-100 text-emerald-700",
  };
  return (
    <span className={`px-2.5 py-0.5 rounded-full text-xs font-medium ${styles[status] || "bg-gray-100 text-gray-800"}`}>
      {status}
    </span>
  );
};

const MetricCard = ({ title, value, subtext, trend, trendUp }) => (
  <Card className="p-5">
    <div className="flex justify-between items-start">
      <div>
        <p className="text-sm font-medium text-slate-500">{title}</p>
        <h3 className="text-2xl font-bold text-slate-900 mt-1">{value}</h3>
      </div>
      <div className={`p-2 rounded-lg ${trendUp ? 'bg-emerald-50' : 'bg-rose-50'}`}>
        {trendUp ? <ArrowUpRight className="w-5 h-5 text-emerald-600" /> : <ArrowDownRight className="w-5 h-5 text-rose-600" />}
      </div>
    </div>
    <div className="mt-4 flex items-center text-xs">
      <span className={`font-medium ${trendUp ? 'text-emerald-600' : 'text-rose-600'}`}>
        {trend}
      </span>
      <span className="text-slate-400 ml-1">{subtext}</span>
    </div>
  </Card>
);

// --- Modal Component ---
const Modal = ({ isOpen, onClose, title, children }) => {
  if (!isOpen) return null;
  return (
    <div className="fixed inset-0 z-50 flex items-center justify-center bg-slate-900/50 backdrop-blur-sm p-4 animate-fade-in">
      <div className="bg-white rounded-xl shadow-xl w-full max-w-lg max-h-[90vh] overflow-y-auto">
        <div className="flex justify-between items-center p-4 border-b border-slate-100 sticky top-0 bg-white z-10">
          <h3 className="font-bold text-lg text-slate-800">{title}</h3>
          <button onClick={onClose} className="text-slate-400 hover:text-slate-600">
            <X className="w-5 h-5" />
          </button>
        </div>
        <div className="p-6">
          {children}
        </div>
      </div>
    </div>
  );
};

// --- Inventory Form Component ---
const InventoryForm = ({ initialData = {}, onSubmit, onCancel }) => {
  const [formData, setFormData] = useState({
    name: initialData.name || '',
    category: initialData.category || '',
    warehouse: initialData.stock?.warehouse || 0,
    store: initialData.stock?.store || 0,
    safetyStock: initialData.safetyStock || 0,
    cost: initialData.cost || 0,
    price: initialData.price || 0,
    vendor: initialData.vendor || ''
  });

  const handleChange = (e) => {
    const { name, value } = e.target;
    setFormData(prev => ({
      ...prev,
      [name]: ['warehouse', 'store', 'safetyStock', 'cost', 'price'].includes(name) ? parseFloat(value) || 0 : value
    }));
  };

  const handleSubmit = (e) => {
    e.preventDefault();
    onSubmit(formData);
  };

  const InputField = ({ label, name, type = "text", step }) => (
    <div className="space-y-1">
      <label className="text-xs font-medium text-slate-500 uppercase tracking-wider">{label}</label>
      <input
        type={type}
        name={name}
        step={step}
        value={formData[name]}
        onChange={handleChange}
        className="w-full px-3 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
        required
      />
    </div>
  );

  return (
    <form onSubmit={handleSubmit} className="space-y-4">
      <InputField label="Product Name" name="name" />
      <div className="grid grid-cols-2 gap-4">
        <InputField label="Category" name="category" />
        <InputField label="Vendor" name="vendor" />
      </div>
      <div className="grid grid-cols-2 gap-4">
        <InputField label="Cost ($)" name="cost" type="number" step="0.01" />
        <InputField label="Price ($)" name="price" type="number" step="0.01" />
      </div>
      <div className="p-4 bg-slate-50 rounded-lg space-y-4 border border-slate-100">
        <h4 className="text-sm font-bold text-slate-700">Stock Levels</h4>
        <div className="grid grid-cols-3 gap-3">
          <InputField label="Warehouse" name="warehouse" type="number" />
          <InputField label="Store" name="store" type="number" />
          <InputField label="Safety Stock" name="safetyStock" type="number" />
        </div>
      </div>

      <div className="flex justify-end gap-3 pt-4 border-t border-slate-100">
        <button type="button" onClick={onCancel} className="px-4 py-2 text-sm font-medium text-slate-600 bg-white border border-slate-200 rounded-lg hover:bg-slate-50">Cancel</button>
        <button type="submit" className="px-4 py-2 text-sm font-medium text-white bg-slate-900 rounded-lg hover:bg-slate-800">Save Changes</button>
      </div>
    </form>
  );
};

// --- Main Application ---

export default function App() {
  const [activeTab, setActiveTab] = useState('dashboard');
  const [isMobileMenuOpen, setIsMobileMenuOpen] = useState(false);

  // App State
  const [inventory, setInventory] = useState(INITIAL_INVENTORY);
  const [orders, setOrders] = useState(INITIAL_ORDERS);
  const [purchaseOrders, setPurchaseOrders] = useState(INITIAL_PO);
  const [searchTerm, setSearchTerm] = useState('');

  // Modal State
  const [isAddModalOpen, setIsAddModalOpen] = useState(false);
  const [editingItem, setEditingItem] = useState(null);

  // Derived Metrics
  const totalStockValue = useMemo(() =>
    inventory.reduce((acc, item) => acc + ((item.stock.warehouse + item.stock.store) * item.cost), 0),
  [inventory]);

  const lowStockCount = useMemo(() =>
    inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).length,
  [inventory]);

  const pendingOrdersCount = useMemo(() =>
    orders.filter(o => o.status === 'Pending').length,
  [orders]);

  // Actions
  const fulfillOrder = (orderId) => {
    setOrders(orders.map(o => o.id === orderId ? { ...o, status: 'Shipped' } : o));
    alert(`Order ${orderId} marked as Shipped. (Simulation: Inventory would be adjusted here)`);
  };

  const receivePO = (poId) => {
    setPurchaseOrders(purchaseOrders.map(p => p.id === poId ? { ...p, status: 'Received' } : p));
    alert(`PO ${poId} received. (Simulation: Stock levels would be updated here)`);
  };

  const handleExport = () => {
    const headers = ['ID,Name,Category,Warehouse Stock,Store Stock,Safety Stock,Cost,Price,Vendor'];
    const rows = inventory.map(item => {
      return [
        item.id,
        `"${item.name}"`,
        item.category,
        item.stock.warehouse,
        item.stock.store,
        item.safetyStock,
        item.cost.toFixed(2),
        item.price.toFixed(2),
        item.vendor
      ].join(',');
    });
    const csvString = [headers, ...rows].join('\n');
    const blob = new Blob([csvString], { type: 'text/csv' });
    const url = URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = 'inventory_export.csv';
    document.body.appendChild(a);
    a.click();
    document.body.removeChild(a);
    URL.revokeObjectURL(url);
  };

  const handleInventorySubmit = (data) => {
    const itemData = {
      name: data.name,
      category: data.category,
      stock: {
        warehouse: data.warehouse,
        store: data.store
      },
      safetyStock: data.safetyStock,
      cost: data.cost,
      price: data.price,
      vendor: data.vendor
    };

    if (editingItem) {
      // Update existing item
      setInventory(prevInventory => prevInventory.map(item =>
        item.id === editingItem.id
          ? { ...item, ...itemData }
          : item
      ));
      alert(`SKU ${editingItem.id} updated successfully.`);
    } else {
      // Add new item
      const newItem = {
        id: `SKU-${Math.floor(Math.random() * 10000)}`,
        ...itemData
      };
      setInventory(prevInventory => [...prevInventory, newItem]);
      alert(`New SKU ${newItem.id} added successfully.`);
    }

    // Close modals and reset state
    setIsAddModalOpen(false);
    setEditingItem(null);
  };

  const filteredInventory = inventory.filter(item =>
    item.name.toLowerCase().includes(searchTerm.toLowerCase()) ||
    item.id.toLowerCase().includes(searchTerm.toLowerCase())
  );

  const Navigation = () => (
    <nav className="space-y-1">
      {[
        { id: 'dashboard', label: 'Dashboard', icon: LayoutDashboard },
        { id: 'inventory', label: 'Inventory', icon: Package },
        { id: 'orders', label: 'Orders', icon: ShoppingCart },
        { id: 'purchasing', label: 'Purchasing', icon: Truck },
        { id: 'reports', label: 'Reports', icon: BarChart3 },
        { id: 'settings', label: 'Settings', icon: Settings },
      ].map((item) => (
        <button
          key={item.id}
          onClick={() => {
            setActiveTab(item.id);
            setIsMobileMenuOpen(false);
          }}
          className={`w-full flex items-center px-4 py-3 text-sm font-medium rounded-lg transition-colors ${
            activeTab === item.id
              ? 'bg-slate-900 text-white'
              : 'text-slate-600 hover:bg-slate-100'
          }`}
        >
          <item.icon className="w-5 h-5 mr-3" />
          {item.label}
          {item.id === 'orders' && pendingOrdersCount > 0 && (
            <span className="ml-auto bg-rose-500 text-white text-xs font-bold px-2 py-0.5 rounded-full">
              {pendingOrdersCount}
            </span>
          )}
        </button>
      ))}
    </nav>
  );

  const renderContent = () => {
    switch(activeTab) {
      case 'dashboard':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4">
              <MetricCard
                title="Total Stock Value"
                value={`$${totalStockValue.toLocaleString()}`}
                trend="+12%"
                subtext="vs last month"
                trendUp={true}
              />
              <MetricCard
                title="Pending Orders"
                value={pendingOrdersCount}
                trend="-5%"
                subtext="processing faster"
                trendUp={true}
              />
              <MetricCard
                title="Low Stock SKUs"
                value={lowStockCount}
                trend="+2"
                subtext="needs attention"
                trendUp={false}
              />
              <MetricCard
                title="Gross Margin (Avg)"
                value="68%"
                trend="+1.2%"
                subtext="healthy"
                trendUp={true}
              />
            </div>

            <div className="grid grid-cols-1 lg:grid-cols-3 gap-6">
              <Card className="col-span-2 p-6">
                <div className="flex justify-between items-center mb-6">
                  <h3 className="font-bold text-slate-800">Recent Sales Velocity</h3>
                  <select className="text-sm border-slate-200 rounded-md text-slate-500">
                    <option>Last 7 Days</option>
                  </select>
                </div>
                {/* Simulated Chart */}
                <div className="h-64 flex items-end space-x-2">
                  {[45, 60, 35, 70, 85, 60, 75].map((h, i) => (
                    <div key={i} className="flex-1 bg-slate-100 rounded-t-lg relative group">
                      <div
                        className="absolute bottom-0 w-full bg-slate-800 rounded-t-lg transition-all duration-500 hover:bg-indigo-600"
                        style={{ height: `${h}%` }}
                      ></div>
                      <div className="opacity-0 group-hover:opacity-100 absolute -top-8 left-1/2 -translate-x-1/2 bg-slate-900 text-white text-xs py-1 px-2 rounded">
                        ${h * 100}
                      </div>
                    </div>
                  ))}
                </div>
                <div className="flex justify-between mt-2 text-xs text-slate-400">
                  <span>Mon</span><span>Tue</span><span>Wed</span><span>Thu</span><span>Fri</span><span>Sat</span><span>Sun</span>
                </div>
              </Card>

              <Card className="p-6">
                <h3 className="font-bold text-slate-800 mb-4">Stock Alerts</h3>
                <div className="space-y-4">
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) < i.safetyStock).map(item => (
                    <div key={item.id} className="flex items-start p-3 bg-red-50 rounded-lg border border-red-100">
                      <AlertCircle className="w-5 h-5 text-red-500 mt-0.5 mr-3 flex-shrink-0" />
                      <div>
                        <p className="text-sm font-medium text-red-900">{item.name}</p>
                        <p className="text-xs text-red-700 mt-1">
                          Only {item.stock.warehouse + item.stock.store} units left (Safety: {item.safetyStock})
                        </p>
                        <button className="mt-2 text-xs font-medium text-red-600 hover:text-red-800">
                          Create PO &rarr;
                        </button>
                      </div>
                    </div>
                  ))}
                  {inventory.filter(i => (i.stock.warehouse + i.stock.store) >= i.safetyStock).slice(0, 2).map(item => (
                    <div key={item.id} className="flex items-center p-3 hover:bg-slate-50 rounded-lg transition-colors">
                      <div className="w-2 h-2 rounded-full bg-emerald-500 mr-3"></div>
                      <div className="flex-1">
                        <p className="text-sm font-medium text-slate-700">{item.name}</p>
                        <p className="text-xs text-slate-500">{item.stock.warehouse + item.stock.store} Units Available</p>
                      </div>
                    </div>
                  ))}
                </div>
              </Card>
            </div>
          </div>
        );

      case 'inventory':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex flex-col sm:flex-row justify-between items-start sm:items-center gap-4">
              <h2 className="text-xl font-bold text-slate-900">Inventory Management</h2>
              <div className="flex gap-2">
                <button
                  onClick={handleExport}
                  className="flex items-center px-4 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50"
                >
                  <Download className="w-4 h-4 mr-2" /> Export
                </button>
                <button
                  onClick={() => setIsAddModalOpen(true)}
                  className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800"
                >
                  <Plus className="w-4 h-4 mr-2" /> Add SKU
                </button>
              </div>
            </div>

            <Card className="overflow-hidden">
              <div className="p-4 border-b border-slate-100 bg-slate-50/50 flex flex-col sm:flex-row gap-4">
                <div className="relative flex-1">
                  <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
                  <input
                    type="text"
                    placeholder="Search SKUs, names, or categories..."
                    className="w-full pl-10 pr-4 py-2 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-slate-900 focus:border-transparent"
                    value={searchTerm}
                    onChange={(e) => setSearchTerm(e.target.value)}
                  />
                </div>
                <button className="flex items-center px-3 py-2 bg-white border border-slate-200 rounded-lg text-sm font-medium text-slate-600 hover:bg-slate-50">
                  <Filter className="w-4 h-4 mr-2" /> Filters
                </button>
              </div>

              <div className="overflow-x-auto">
                <table className="w-full text-left text-sm">
                  <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                    <tr>
                      <th className="px-6 py-4">Product / SKU</th>
                      <th className="px-6 py-4">Location Split</th>
                      <th className="px-6 py-4 text-right">Cost</th>
                      <th className="px-6 py-4 text-right">Price</th>
                      <th className="px-6 py-4 text-right">Margin</th>
                      <th className="px-6 py-4">Status</th>
                      <th className="px-6 py-4 text-right">Actions</th>
                    </tr>
                  </thead>
                  <tbody className="divide-y divide-slate-100">
                    {filteredInventory.map((item) => {
                      const total = item.stock.warehouse + item.stock.store;
                      const margin = ((item.price - item.cost) / item.price * 100).toFixed(0);
                      return (
                        <tr key={item.id} className="hover:bg-slate-50/50 transition-colors">
                          <td className="px-6 py-4">
                            <div className="font-medium text-slate-900">{item.name}</div>
                            <div className="text-xs text-slate-400 font-mono mt-0.5">{item.id}</div>
                          </td>
                          <td className="px-6 py-4">
                            <div className="flex flex-col gap-1 text-xs">
                              <span className="flex justify-between w-32"><span className="text-slate-500">Wholesale:</span> <span className="font-mono">{item.stock.warehouse}</span></span>
                              <span className="flex justify-between w-32"><span className="text-slate-500">Store:</span> <span className="font-mono">{item.stock.store}</span></span>
                              <div className="w-32 h-1 bg-slate-100 rounded-full mt-1 overflow-hidden">
                                <div className="h-full bg-slate-900" style={{ width: `${(total / (item.safetyStock * 3)) * 100}%` }}></div>
                              </div>
                            </div>
                          </td>
                          <td className="px-6 py-4 text-right font-mono text-slate-600">${item.cost.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right font-mono text-slate-900">${item.price.toFixed(2)}</td>
                          <td className="px-6 py-4 text-right">
                            <span className={`inline-block px-2 py-0.5 rounded text-xs font-medium ${margin > 50 ? 'bg-emerald-100 text-emerald-700' : 'bg-amber-100 text-amber-700'}`}>
                              {margin}%
                            </span>
                          </td>
                          <td className="px-6 py-4">
                            <Badge status={total < item.safetyStock ? 'Low' : 'Normal'} />
                          </td>
                          <td className="px-6 py-4 text-right text-slate-400">
                            <button
                              onClick={() => setEditingItem(item)}
                              className="hover:text-slate-900"
                            >
                              Edit
                            </button>
                          </td>
                        </tr>
                      );
                    })}
                  </tbody>
                </table>
              </div>
            </Card>

            {/* Add SKU Modal */}
            <Modal
              isOpen={isAddModalOpen}
              onClose={() => setIsAddModalOpen(false)}
              title="Add New SKU"
            >
              <InventoryForm
                onSubmit={handleInventorySubmit}
                onCancel={() => setIsAddModalOpen(false)}
              />
            </Modal>

            {/* Edit SKU Modal */}
            <Modal
              isOpen={!!editingItem}
              onClose={() => setEditingItem(null)}
              title="Edit SKU"
            >
              {editingItem && (
                <InventoryForm
                  initialData={editingItem}
                  onSubmit={handleInventorySubmit}
                  onCancel={() => setEditingItem(null)}
                />
              )}
            </Modal>
          </div>
        );

      case 'orders':
        return (
          <div className="space-y-6 animate-fade-in">
             <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">All Orders</h2>
              <div className="text-sm text-slate-500">Syncing with Shopify & Amazon...</div>
            </div>
            <Card className="overflow-hidden">
              <table className="w-full text-left text-sm">
                <thead className="bg-slate-50 text-slate-500 font-medium border-b border-slate-100">
                  <tr>
                    <th className="px-6 py-4">Order ID</th>
                    <th className="px-6 py-4">Customer</th>
                    <th className="px-6 py-4">Channel</th>
                    <th className="px-6 py-4">Date</th>
                    <th className="px-6 py-4 text-right">Total</th>
                    <th className="px-6 py-4">Status</th>
                    <th className="px-6 py-4 text-right">Action</th>
                  </tr>
                </thead>
                <tbody className="divide-y divide-slate-100">
                  {orders.map((order) => (
                    <tr key={order.id} className="hover:bg-slate-50/50">
                      <td className="px-6 py-4 font-mono font-medium text-slate-900">{order.id}</td>
                      <td className="px-6 py-4">{order.customer}</td>
                      <td className="px-6 py-4">
                        <span className="flex items-center gap-2">
                           <span className={`w-2 h-2 rounded-full ${order.channel === 'Shopify' ? 'bg-green-500' : order.channel === 'Amazon' ? 'bg-orange-500' : 'bg-blue-500'}`}></span>
                           {order.channel}
                        </span>
                      </td>
                      <td className="px-6 py-4 text-slate-500">{order.date}</td>
                      <td className="px-6 py-4 text-right font-medium">${order.total.toFixed(2)}</td>
                      <td className="px-6 py-4"><Badge status={order.status} /></td>
                      <td className="px-6 py-4 text-right">
                        {order.status === 'Pending' && (
                          <button
                            onClick={() => fulfillOrder(order.id)}
                            className="text-xs bg-slate-900 text-white px-3 py-1.5 rounded-md hover:bg-slate-700 transition-colors"
                          >
                            Ship
                          </button>
                        )}
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </Card>
          </div>
        );

      case 'purchasing':
        return (
          <div className="space-y-6 animate-fade-in">
            <div className="flex justify-between items-center">
              <h2 className="text-xl font-bold text-slate-900">Purchase Orders</h2>
              <button className="flex items-center px-4 py-2 bg-slate-900 text-white rounded-lg text-sm font-medium hover:bg-slate-800">
                <Plus className="w-4 h-4 mr-2" /> New PO
              </button>
            </div>
             <div className="grid gap-6">
                {purchaseOrders.map((po) => (
                  <Card key={po.id} className="p-0 overflow-hidden flex flex-col md:flex-row">
                     <div className="p-6 flex-1">
                        <div className="flex justify-between items-start mb-4">
                          <div>
                            <div className="flex items-center gap-3">
                              <h3 className="font-bold text-lg text-slate-900">{po.id}</h3>
                              <Badge status={po.status} />
                            </div>
                            <p className="text-slate-500 text-sm mt-1">Vendor: <span className="font-medium text-slate-700">{po.vendor}</span></p>
                          </div>
                          <div className="text-right">
                             <div className="font-mono font-bold text-lg">${po.total.toFixed(2)}</div>
                             <div className="text-xs text-slate-400">Total Value</div>
                          </div>
                        </div>
                        <div className="flex items-center gap-6 text-sm text-slate-600">
                           <div className="flex items-center gap-2">
                              <Truck className="w-4 h-4" />
                              <span>Expected: {po.expected}</span>
                           </div>
                           <div className="flex items-center gap-2">
                              <DollarSign className="w-4 h-4" />
                              <span>Payment: Net 30</span>
                           </div>
                        </div>
                     </div>
                     {po.status !== 'Received' && (
                       <div className="bg-slate-50 p-6 flex items-center border-t md:border-t-0 md:border-l border-slate-100">
                          <button
                            onClick={() => receivePO(po.id)}
                            className="w-full md:w-auto flex items-center justify-center px-6 py-2 bg-emerald-600 text-white rounded-lg hover:bg-emerald-700 transition-colors font-medium text-sm"
                          >
                            <CheckCircle className="w-4 h-4 mr-2" />
                            Receive Goods
                          </button>
                       </div>
                     )}
                  </Card>
                ))}
             </div>
          </div>
        );

      default:
        return <div className="p-10 text-center text-slate-400">Module under construction</div>;
    }
  };

  return (
    <div className="flex h-screen bg-slate-50 font-sans text-slate-900 overflow-hidden">
      {/* Added style block to define the animation which was missing in Tailwind default config */}
      <style>{`
        @keyframes fadeIn {
          from { opacity: 0; transform: translateY(10px); }
          to { opacity: 1; transform: translateY(0); }
        }
        .animate-fade-in {
          animation: fadeIn 0.4s ease-out forwards;
        }
      `}</style>

      {/* Sidebar for Desktop */}
      <aside className="hidden md:flex flex-col w-64 bg-white border-r border-slate-200 h-full">
        <div className="p-6 flex items-center border-b border-slate-100">
          <div className="w-8 h-8 bg-indigo-600 rounded-lg flex items-center justify-center mr-3 shadow-sm">
            <Package className="w-5 h-5 text-white" />
          </div>
          <span className="font-bold text-lg tracking-tight">ZeroERP</span>
        </div>
        <div className="flex-1 overflow-y-auto py-6 px-4">
          <Navigation />
        </div>
        <div className="p-4 border-t border-slate-100">
          <div className="flex items-center p-2 rounded-lg bg-slate-50 border border-slate-100">
            <div className="w-8 h-8 rounded-full bg-slate-200 flex items-center justify-center text-xs font-bold text-slate-600">
              JS
            </div>
            <div className="ml-3">
              <p className="text-xs font-bold text-slate-900">John Smith</p>
              <p className="text-[10px] text-slate-500">Admin</p>
            </div>
          </div>
        </div>
      </aside>

      {/* Mobile Sidebar Overlay */}
      {isMobileMenuOpen && (
        <div className="fixed inset-0 z-50 bg-slate-900/50 md:hidden" onClick={() => setIsMobileMenuOpen(false)}>
          <div className="absolute left-0 top-0 bottom-0 w-64 bg-white p-4" onClick={e => e.stopPropagation()}>
             <div className="flex justify-between items-center mb-6">
                <span className="font-bold text-lg">ZeroERP</span>
                <button onClick={() => setIsMobileMenuOpen(false)}><X className="w-6 h-6" /></button>
             </div>
             <Navigation />
          </div>
        </div>
      )}

      {/* Main Content Area */}
      <main className="flex-1 flex flex-col h-full overflow-hidden">
        {/* Top Header */}
        <header className="h-16 bg-white border-b border-slate-200 flex justify-between items-center px-6">
          <div className="flex items-center md:hidden">
            <button onClick={() => setIsMobileMenuOpen(true)} className="mr-4">
              <Menu className="w-6 h-6 text-slate-600" />
            </button>
          </div>

          <div className="hidden md:flex flex-1 max-w-md relative">
            <Search className="absolute left-3 top-1/2 -translate-y-1/2 w-4 h-4 text-slate-400" />
            <input
              type="text"
              placeholder="Command + K to search..."
              className="w-full pl-10 pr-4 py-2 bg-slate-50 border border-slate-200 rounded-lg text-sm focus:outline-none focus:ring-2 focus:ring-indigo-500/20 focus:border-indigo-500 transition-all"
            />
            <div className="absolute right-3 top-1/2 -translate-y-1/2 flex gap-1">
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">Cmd</span>
               <span className="px-1.5 py-0.5 rounded border border-slate-200 bg-white text-[10px] text-slate-400 font-medium">K</span>
            </div>
          </div>

          <div className="flex items-center gap-4">
            <button className="relative p-2 text-slate-400 hover:bg-slate-50 rounded-lg transition-colors">
              <Bell className="w-5 h-5" />
              <span className="absolute top-2 right-2 w-2 h-2 bg-rose-500 rounded-full border-2 border-white"></span>
            </button>
          </div>
        </header>

        {/* Scrollable Page Content */}
        <div className="flex-1 overflow-auto p-4 md:p-8">
           <div className="max-w-7xl mx-auto">
             {renderContent()}
           </div>
        </div>
      </main>
    </div>
  );
}


Overwriting ZeroERP.js


## Final Task

### Subtask:
Verify the implementation and facilitate the download of the updated application.


## Summary:

### Data Analysis Key Findings

*   **CSV Export Implementation**: Successfully integrated a `handleExport` function that generates a downloadable file named `inventory_export.csv`. The logic correctly maps the complex state object (including nested properties like `item.stock.warehouse`) to a comma-separated string with headers: ID, Name, Category, Warehouse Stock, Store Stock, Safety Stock, Cost, Price, and Vendor.
*   **Component Architecture**: Developed modular `Modal` and `InventoryForm` components. The form handles local state for inputs (Name, Category, Vendor, Cost, Price) and specifically manages the separate warehouse and store stock integers.
*   **State Management & Logic**:
    *   Implemented `handleInventorySubmit` to manage state updates.
    *   The logic automatically distinguishes between **creating** a new item (generating a random ID format `SKU-XXXX`) and **updating** an existing item based on the presence of `editingItem`.
    *   Flat form data is successfully restructured back into the required nested `stock` object format before saving to the main `inventory` state.

### Insights or Next Steps

*   **Data Persistence**: Currently, all inventory changes (Add/Edit) are stored in the local React state and will be lost upon page refresh. A necessary next step is connecting these actions to a backend API or LocalStorage to ensure data persistence.
*   **Validation & Deletion**: The current implementation allows for creation and updates but lacks a "Delete" function. Additionally, implementing stricter input validation (e.g., preventing negative stock numbers or duplicate SKU IDs) would enhance data integrity.
